# How to Automate IV: Automate your own Model

---

*This is an archived version of How to Automate 4* the latest version is available [here](../How_to_Automate_4)

For this tutorial we will be automating the model that Bruno taught us how to make in the [Greyhound Modelling Tutorial](https://betfair-datascientists.github.io/modelling/greyhoundModellingPython/). This tutorial follows on logically from [How to Automate III](../How_to_Automate_3). If you haven't already, make sure you take a look at the [rest of the series](../How_to_Automate_1) first those before continuing here as they cover some key concepts!

---

## Saving and loading in our model

To generate our predictions, we have two options: we can generate our predictions using the same notebook used to train our model then read those predictions into this notebook, or we can save the model and read that model into this notebook.

For this tutorial we have chosen to save the model, as it becomes a bit less confusing and easier to manage, although there are some pieces of code we may have to write twice (copy and paste). So first we will need to run the code from the tutorial and then save the model. This is super as simple we can just copy and paste the complete code provided at the [end of the tutorial](https://betfair-datascientists.github.io/modelling/greyhoundModellingPython/#complete-code) or [download from Github](https://github.com/betfair-down-under/autoHubTutorials/tree/master/greyhoundModelling). Then we can just run this extra line code (which I have copied from the [documentation page](https://scikit-learn.org/stable/modules/model_persistence.html)) at the end of the notebook to save the model. 

```
from joblib import dump
dump(models['LogisticRegression'], 'logistic_regression.joblib')
```

Now that the file is saved, let's read it into this note book:

In [1]:
from joblib import load

brunos_model = load('logistic_regression.joblib')
brunos_model

LogisticRegression(n_jobs=-1, solver='saga')

---

## Generating predictions for today

Now that we have the model loaded in, we need the data, to generate our predictions for today's races!

In [2]:
# Import libraries required to download today's races
import os
import sys

# Allow imports from src folder
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil import tz
from pandas.tseries.offsets import MonthEnd
from sklearn.preprocessing import MinMaxScaler
import itertools

import numpy as np
import pandas as pd
from nltk.tokenize import regexp_tokenize

# settings to display all columns
pd.set_option("display.max_columns", None)

import fasttrack as ft

from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Validate FastTrack API connection
api_key = os.getenv('FAST_TRACK_API_KEY',)
client = ft.Fasttrack(api_key)
track_codes = client.listTracks()

Valid Security Key


In [4]:
# Import race data excluding NZ races
au_tracks_filter = list(track_codes[track_codes['state'] != 'NZ']['track_code'])

# Time window to import data
# First day of the month 46 months back from now
date_from = (datetime.today() - relativedelta(months=46)).replace(day=1).strftime('%Y-%m-%d')
# First day of previous month
date_to = (datetime.today() - relativedelta(months=1)).replace(day=1).strftime('%Y-%m-%d')

# Dataframes to populate data with
race_details = pd.DataFrame()
dog_results = pd.DataFrame()

# For each month, either fetch data from API or use local CSV file if we already have downloaded it
for start in pd.date_range(date_from, date_to, freq='MS'):
    start_date = start.strftime("%Y-%m-%d")
    end_date = (start + MonthEnd(1)).strftime("%Y-%m-%d")
    try:
        filename_races = f'FT_AU_RACES_{start_date}.csv'
        filename_dogs = f'FT_AU_DOGS_{start_date}.csv'

        filepath_races = f'../data/{filename_races}'
        filepath_dogs = f'../data/{filename_dogs}'

        print(f'Loading data from {start_date} to {end_date}')
        if os.path.isfile(filepath_races):
            # Load local CSV file
            month_race_details = pd.read_csv(filepath_races) 
            month_dog_results = pd.read_csv(filepath_dogs) 
        else:
            # Fetch data from API
            month_race_details, month_dog_results = client.getRaceResults(start_date, end_date, au_tracks_filter)
            month_race_details.to_csv(filepath_races, index=False)
            month_dog_results.to_csv(filepath_dogs, index=False)

        # Combine monthly data
        race_details = race_details.append(month_race_details, ignore_index=True)
        dog_results = dog_results.append(month_dog_results, ignore_index=True)
    except:
        print(f'Could not load data from {start_date} to {end_date}')

Loading data from 2018-09-01 to 2018-09-30
Loading data from 2018-10-01 to 2018-10-31
Loading data from 2018-11-01 to 2018-11-30
Loading data from 2018-12-01 to 2018-12-31
Loading data from 2019-01-01 to 2019-01-31
Loading data from 2019-02-01 to 2019-02-28
Loading data from 2019-03-01 to 2019-03-31
Loading data from 2019-04-01 to 2019-04-30
Loading data from 2019-05-01 to 2019-05-31
Loading data from 2019-06-01 to 2019-06-30
Loading data from 2019-07-01 to 2019-07-31
Loading data from 2019-08-01 to 2019-08-31
Loading data from 2019-09-01 to 2019-09-30
Loading data from 2019-10-01 to 2019-10-31
Loading data from 2019-11-01 to 2019-11-30
Loading data from 2019-12-01 to 2019-12-31
Loading data from 2020-01-01 to 2020-01-31
Loading data from 2020-02-01 to 2020-02-29
Loading data from 2020-03-01 to 2020-03-31
Loading data from 2020-04-01 to 2020-04-30
Loading data from 2020-05-01 to 2020-05-31
Loading data from 2020-06-01 to 2020-06-30
Loading data from 2020-07-01 to 2020-07-31
Loading dat

c:\Users\zhoui\greyhounds_bruno\greyhound-modelling\venv_greyhounds\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading data from 2022-02-01 to 2022-02-28
Loading data from 2022-03-01 to 2022-03-31
Loading data from 2022-04-01 to 2022-04-30
Loading data from 2022-05-01 to 2022-05-31
Loading data from 2022-06-01 to 2022-06-30


This piece of code we copied and pasted from the [Greyhound Modelling Tutorial](https://betfair-datascientists.github.io/modelling/greyhoundModellingPython/) is fantastic! It has downloaded/read-in a ton of historic data! There is an issue though! We don't have the data for today's races, and also for any races that has occurred this month. This is because the code above only downloaded data up until the end of last month. 

For example, if we are in the middle of June, then any races in the first two weeks of June won't be downloaded by the chunk of code above. An issue is that if we download it now, when tomorrow rolls around it won't include the extra races that have finished today. 

So, the simple but inefficient solution is that every single day we redownload all the races that have already concluded this month. (Ideally you have some sort of database set up or you store and download your data in a daily format instead of the monthly format)

In [5]:
race_details.tail()

,@id,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date
88510,792243395,6,SKY RACING (N/P) STAKE,01:27PM,300m,Restricted Win,Murray Bridge (MBS),07 Jun 22
88511,792243396,7,KURT DONSBERG PHOTOGRAPHY MIXED STAKE,01:44PM,300m,Mixed 4/5,Murray Bridge (MBS),07 Jun 22
88512,792243397,8,GREYHOUNDS AS PETS,02:04PM,300m,Grade 5 Final,Murray Bridge (MBS),07 Jun 22
88513,792243398,9,@THEDOGSSA (N/P) STAKE,02:19PM,300m,Restricted Win,Murray Bridge (MBS),07 Jun 22
88514,792243399,10,FOLLOW THEDOGSSA ON TWITTER (N/P) STAKE,02:39PM,300m,Restricted Win,Murray Bridge (MBS),07 Jun 22


In [6]:
current_month_start_date = pd.Timestamp.now().replace(day=1).strftime("%Y-%m-%d")
current_month_end_date = (pd.Timestamp.now().replace(day=1)+ MonthEnd(1))
current_month_end_date = (current_month_end_date - pd.Timedelta('1 day')).strftime("%Y-%m-%d")

print(f'Start date: {current_month_start_date}')
print(f'End Date: {current_month_end_date}')

Start date: 2022-07-01
End Date: 2022-07-30


In [7]:
# Download data for races that have concluded this current month up untill today
# Start and end dates for current month
current_month_start_date = pd.Timestamp.now().replace(day=1).strftime("%Y-%m-%d")
current_month_end_date = (pd.Timestamp.now().replace(day=1)+ MonthEnd(1))
current_month_end_date = (current_month_end_date - pd.Timedelta('1 day')).strftime("%Y-%m-%d")

# Files names 
filename_races = f'FT_AU_RACES_{current_month_start_date}.csv'
filename_dogs = f'FT_AU_DOGS_{current_month_start_date}.csv'
# Where to store files locally
filepath_races = f'../data/{filename_races}'
filepath_dogs = f'../data/{filename_dogs}'

# Fetch data from API
month_race_details, month_dog_results = client.getRaceResults(current_month_start_date, current_month_end_date, au_tracks_filter)

# Save the files locally and replace any out of date fields
month_race_details.to_csv(filepath_races, index=False)
month_dog_results.to_csv(filepath_dogs, index=False)

Getting meets for each date ..


100%|██████████| 30/30 [00:14<00:00,  2.01it/s]


Getting historic results details ..


100%|██████████| 162/162 [01:30<00:00,  1.80it/s]


In [8]:
dog_results

,@id,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName
0,114215500,1,DR. MURPHY,7.0,10,29.7,$4.10,NaN,4.24,NaN,Q/111,0,NaN,4.70,22.84,NaN,356387352,107925,W McMahon
1,131737955,2,MOLLY SPOLLY,8.0,8,27.3,$2.20F,NaN,4.24,4.24,M/222,0,NaN,4.72,23.14,NaN,356387352,199516,K Leviston
2,204414097,3,ASTON NARITA,2.0,2,29.2,$4.50,NaN,4.94,0.70,M/343,2,NaN,4.88,23.19,NaN,356387352,101224,K Gorman
3,126744995,4,ONI,6.0,6,25.0,$15.50,NaN,5.70,0.76,S/674,0,NaN,4.95,23.24,NaN,356387352,107925,W McMahon
4,120958941,5,DARCON FLASH,1.0,1,29.3,$31.00,NaN,6.54,0.84,M/765,8,NaN,4.96,23.30,NaN,356387352,125087,R Conway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749514,415996416,1,WEBLEC WHIRL,2.0,2,27.6,$4.20,NaN,0.10,NaN,0,0,NaN,4.38,16.75,510.0,792243399,76598,N Loechel
749515,557002281,2,UP THERE BILLY,1.0,1,32.7,$1.80F,NaN,0.10,0.14,NaN,0,NaN,4.43,16.76,175.0,792243399,327728,J Trengove
749516,529022935,3,WEBLEC FLAME,6.0,6,31.8,$5.50,NaN,4.25,4.00,NaN,0,NaN,4.48,17.04,140.0,792243399,76598,N Loechel
749517,383604709,4,STRAIGHT BLAZE,8.0,8,34.7,$23.00,NaN,5.00,0.86,NaN,0,NaN,4.61,17.10,115.0,792243399,123529,D Johnstone


In [9]:
# This is super important I have spent literally hours before I found out this was causing errors
dog_results['@id'] = pd.to_numeric(dog_results['@id'])

In [10]:
# Append the extra data to our data frames 
race_details = race_details.append(month_race_details, ignore_index=True)
dog_results = dog_results.append(month_dog_results, ignore_index=True)

What we are really interested in are races that are scheduled for today as we want to use our model to predict their ratings. So, let's write some code we can run in the morning that will download the data for the day:

In [11]:
# Download the data for todays races
todays_date = pd.Timestamp.now().strftime("%Y-%m-%d")
todays_races, todays_dogs = client.getFullFormat(dt= todays_date, tracks = au_tracks_filter)

display(todays_races.head(1), todays_dogs.head(1))

Getting meets for each date ..


100%|██████████| 1/1 [00:00<00:00,  1.89it/s]


Getting dog lineups ..


100%|██████████| 12/12 [00:13<00:00,  1.14s/it]


,@id,RaceNum,RaceName,RaceTime,RaceTimeDateUTC,Distance,RaceGrade,PrizeMoney1,PrizeMoney2,PrizeMoney3,PrizeMoney4,PrizeMoney5,PrizeMoney6,PrizeMoney7,PrizeMoney8,GOBIS,Hurdle,Handicap,TAB,GradeCode,VICGREYS,RaceComment,Track,Date,Quali,TipsComments_Bet,TipsComments_Tips
0,801896110,1,GPP LASER3300,06:44PM,04 Jul 22 08:44AM,385m,Maiden,$1600,$460,$230,$115,None,None,None,None,None,None,None,TRI/QUIN R/D EXACTA PICK4,M,None,"""KASUMI BERRY (5) is a well bred type and her ...",Shepparton,04 Jul 22,None,"Box Quinella 1,2,4,7 ($10 for 166.67%)","4, 7, 1, 2"


,@id,RaceBox,DogName,BestTime,DogHandicap,Odds,Rating,Speed,DogComment,StartsTOT,StartsTTD,Suburb,Owner,Colour,Sex,Whelped,DogGrade,DogGOBIS,DogPRIZE,AgedPrizeMoney,Form,DamId,DamName,SireId,SireName,TrainerId,TrainerName,RaceId
0,536196758,1,HAVE A SHIRAZ,FSH,None,$4.60,0,None,Dam produced the highly talented Flaming rush,Starts 0-0-0-0,Trk/Dst 0-0-0-0,Heathcote,Paul Ellis,BK,B,03 Dec 19,M,N,0,0,"[None, None, None, None, None]",1550070039,Pepper Shiraz,-710494,Barcia Bale,117228,Jason Formosa,801896110


In [12]:
# It seems that the todays_races dataframe doesn't have the date column, so let's add that on
todays_races['date'] = pd.Timestamp.now().strftime('%d %b %y')
todays_races.head(1)

,@id,RaceNum,RaceName,RaceTime,RaceTimeDateUTC,Distance,RaceGrade,PrizeMoney1,PrizeMoney2,PrizeMoney3,PrizeMoney4,PrizeMoney5,PrizeMoney6,PrizeMoney7,PrizeMoney8,GOBIS,Hurdle,Handicap,TAB,GradeCode,VICGREYS,RaceComment,Track,Date,Quali,TipsComments_Bet,TipsComments_Tips,date
0,801896110,1,GPP LASER3300,06:44PM,04 Jul 22 08:44AM,385m,Maiden,$1600,$460,$230,$115,None,None,None,None,None,None,None,TRI/QUIN R/D EXACTA PICK4,M,None,"""KASUMI BERRY (5) is a well bred type and her ...",Shepparton,04 Jul 22,None,"Box Quinella 1,2,4,7 ($10 for 166.67%)","4, 7, 1, 2",04 Jul 22


In [13]:
# It also seems that in todays_dogs dataframe Box is labeled as RaceBox instead, so let's rename it
# We can also see that there are some specific dogs that have "Res." as a suffix of their name, i.e. they are reserve dogs,
# We will treat this later
todays_dogs = todays_dogs.rename(columns={"RaceBox":"Box"})
todays_dogs.tail(3)

,@id,Box,DogName,BestTime,DogHandicap,Odds,Rating,Speed,DogComment,StartsTOT,StartsTTD,Suburb,Owner,Colour,Sex,Whelped,DogGrade,DogGOBIS,DogPRIZE,AgedPrizeMoney,Form,DamId,DamName,SireId,SireName,TrainerId,TrainerName,RaceId
1061,400500428,5,TEQUILA TALKING,22.63,None,None,100,66.425,None,Starts 58-11-7-12,Trk/Dst 28-6-5-5,Wolffdene,"Fives Alive Synd D Wolff,N Brauer",BE,D,19 Oct 18,4,N,28855,None,"[{'Place': '2nd', 'FormBox': '5', 'Weight': '3...",255840075,Sivamet,-737547,Hostile,93322,Michael Brauer,801490825
1062,525622257,7,REFERRAL,FSTD,None,None,81,61.343,None,Starts 51-4-7-6,Trk/Dst 0-0-0-0,Laidley Heights,"Bad Decisions Synd P O'Reilly,A Pearce,D Henery",BD,D,22 Oct 19,5,N,13945,None,"[{'Place': '5th', 'FormBox': '7', 'Weight': '3...",257880044,Lovelace,792880037,Sh Avatar,313314,Andrew Pearce,801490825
1063,566347962,8,STARDUST DREAMS,NBT,None,None,92,61.271,None,Starts 22-3-4-2,Trk/Dst 2-0-1-0,Park Ridge,Kerri-Lyn Harkness,BK,D,07 Mar 20,5,N,8240,None,"[{'Place': '2nd', 'FormBox': '8', 'Weight': '3...",118703516,Ellie Belles,141317074,My Redeemer,127311,Stephen Woods,801490825


In [14]:
# Appending todays data to this months data
month_dog_results = pd.concat([month_dog_results,todays_dogs],join='outer')[month_dog_results.columns]
month_race_details = pd.concat([month_race_details,todays_races],join='outer')[month_race_details.columns]

# Appending this months data to the rest of our historical data
race_details = race_details.append(month_race_details, ignore_index=True)
dog_results = dog_results.append(month_dog_results, ignore_index=True)

---

## Cleaning our data and feature creation

Originally I thought that since we now that we have all the data we can easily copy and paste the code used in the [greyhound modelling tutorial](https://betfair-datascientists.github.io/modelling/greyhoundModellingPython/) to clean our data and create the features. 

But after staring at weird predictions and spending hours trying to work out why some things weren't working I realised that for the most part we can copy and paste code, but when working with the live data we do need to make a few changes. I'll point them out when we get to it, but the main things that tripped me up is the data types the FastTrack API gives and that we need a system to work around reserve dogs

In [15]:
race_details

,@id,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date
0,356387352,1,RUTTER'S BUTCHERY & POULTRY,05:29PM,395m,Mixed 6/7,Traralgon,01 Sep 18
1,356387359,2,TAB - WE LOVE A BET,05:47PM,395m,Grade 5,Traralgon,01 Sep 18
2,356387358,3,HALEY CONCRETING,06:05PM,395m,Grade 5,Traralgon,01 Sep 18
3,356387355,4,R.W & A.R INGLIS ELECTRICIANS,06:29PM,395m,Free For All,Traralgon,01 Sep 18
4,356387363,5,PRINTMAC,06:45PM,525m,Grade 5,Traralgon,01 Sep 18
...,...,...,...,...,...,...,...,...
89359,801490821,5,SENNACHIE @ STUD - STEVE WHITE,08:13PM,520m,Grade 5 Heat,Albion Park,04 Jul 22
89360,801490822,6,ORSON ALLEN @ METICULOUS LODGE,08:35PM,520m,Grade 5 Heat,Albion Park,04 Jul 22
89361,801490823,7,SKY RACING,08:53PM,600m,Mixed 4/5,Albion Park,04 Jul 22
89362,801490824,8,BORGBET TIPPING SERVICE,09:15PM,520m,Mixed 3/4,Albion Park,04 Jul 22


The first thing that tripped me up was when `FastTrack_DogId` for live data was in a string format, and because everything looks like it works, it took ages to find this error. So, let's make sure we deal with it here using:

```
dog_results['FastTrack_DogId'] = pd.to_numeric(dog_results['FastTrack_DogId'])
```

In [16]:
## Cleanse and normalise the data
# Clean up the race dataset
race_details = race_details.rename(columns = {'@id': 'FastTrack_RaceId'})
race_details['Distance'] = race_details['Distance'].apply(lambda x: int(x.replace("m", "")))
race_details['date_dt'] = pd.to_datetime(race_details['date'], format = '%d %b %y')
# Clean up the dogs results dataset
dog_results = dog_results.rename(columns = {'@id': 'FastTrack_DogId', 'RaceId': 'FastTrack_RaceId'})

# New line of code (rest of this code chunk is copied from bruno's code)
dog_results['FastTrack_DogId'] = pd.to_numeric(dog_results['FastTrack_DogId'])

# Combine dogs results with race attributes
dog_results = dog_results.merge(
    race_details, 
    how = 'left',
    on = 'FastTrack_RaceId'
)

# Convert StartPrice to probability
dog_results['StartPrice'] = dog_results['StartPrice'].apply(lambda x: None if x is None else float(x.replace('$', '').replace('F', '')) if isinstance(x, str) else x)
dog_results['StartPrice_probability'] = (1 / dog_results['StartPrice']).fillna(0)
dog_results['StartPrice_probability'] = dog_results.groupby('FastTrack_RaceId')['StartPrice_probability'].apply(lambda x: x / x.sum())

# Discard entries without results (scratched or did not finish)
dog_results = dog_results[~dog_results['Box'].isnull()]
dog_results['Box'] = dog_results['Box'].astype(int)

# Clean up other attributes
dog_results['RunTime'] = dog_results['RunTime'].astype(float)
dog_results['SplitMargin'] = dog_results['SplitMargin'].astype(float)
dog_results['Prizemoney'] = dog_results['Prizemoney'].astype(float).fillna(0)
dog_results['Place'] = pd.to_numeric(dog_results['Place'].apply(lambda x: x.replace("=", "") if isinstance(x, str) else 0), errors='coerce').fillna(0)
dog_results['win'] = dog_results['Place'].apply(lambda x: 1 if x == 1 else 0)

# Normalise some of the raw values
dog_results['Prizemoney_norm'] = np.log10(dog_results['Prizemoney'] + 1) / 12
dog_results['Place_inv'] = (1 / dog_results['Place']).fillna(0)
dog_results['Place_log'] = np.log10(dog_results['Place'] + 1).fillna(0)
dog_results['RunSpeed'] = (dog_results['RunTime'] / dog_results['Distance']).fillna(0)

In [17]:
## Generate features using raw data
# Calculate median winner time per track/distance
win_results = dog_results[dog_results['win'] == 1]
median_win_time = pd.DataFrame(data=win_results[win_results['RunTime'] > 0].groupby(['Track', 'Distance'])['RunTime'].median()).rename(columns={"RunTime": "RunTime_median"}).reset_index()
median_win_split_time = pd.DataFrame(data=win_results[win_results['SplitMargin'] > 0].groupby(['Track', 'Distance'])['SplitMargin'].median()).rename(columns={"SplitMargin": "SplitMargin_median"}).reset_index()
median_win_time.head()

# Calculate track speed index
median_win_time['speed_index'] = (median_win_time['RunTime_median'] / median_win_time['Distance'])
median_win_time['speed_index'] = MinMaxScaler().fit_transform(median_win_time[['speed_index']])
median_win_time.head()

# Compare dogs finish time with median winner time
dog_results = dog_results.merge(median_win_time, on=['Track', 'Distance'], how='left')
dog_results = dog_results.merge(median_win_split_time, on=['Track', 'Distance'], how='left')

# Normalise time comparison
dog_results['RunTime_norm'] = (dog_results['RunTime_median'] / dog_results['RunTime']).clip(0.9, 1.1)
dog_results['RunTime_norm'] = MinMaxScaler().fit_transform(dog_results[['RunTime_norm']])
dog_results['SplitMargin_norm'] = (dog_results['SplitMargin_median'] / dog_results['SplitMargin']).clip(0.9, 1.1)
dog_results['SplitMargin_norm'] = MinMaxScaler().fit_transform(dog_results[['SplitMargin_norm']])
dog_results.head()

# Calculate box winning percentage for each track/distance
box_win_percent = pd.DataFrame(data=dog_results.groupby(['Track', 'Distance', 'Box'])['win'].mean()).rename(columns={"win": "box_win_percent"}).reset_index()
# Add to dog results dataframe
dog_results = dog_results.merge(box_win_percent, on=['Track', 'Distance', 'Box'], how='left')
# Display example of barrier winning probabilities
print(box_win_percent.head(8))

         Track  Distance  Box  box_win_percent
0  Albion Park       331    1         0.198089
1  Albion Park       331    2         0.152116
2  Albion Park       331    3         0.127354
3  Albion Park       331    4         0.126605
4  Albion Park       331    5         0.111058
5  Albion Park       331    6         0.109304
6  Albion Park       331    7         0.105310
7  Albion Park       331    8         0.115146


The second thing that we need to add is related to reserve dogs, and this took me ages to come to this solution, but if you have a better one, please submit a pull request.


Basically, a single greyhound can be a reserve dog for multiple races on the same day. They each appear as a new row in our data frame. For example, 'MACI REID' is a reserve dog for three different races on the 2022-09-02:

![reserve_dogs_example](hta_img/reserve_dogs_example.png)

When we try lag our data by using `.shift(1)` like in Bruno's original code it will produce the wrong values for our features. In the above example only the first race The Gardens Race 4 (the third row) will have correct data but all the rows under it will have incorrectly calculated features. We need each of the following rows to be the same as the third row. The solution that I have come up with is a little bit complicated, but it gets the job done:

```
# Please submit a pull request if you have a better solution
temp = rolling_result.reset_index()
temp = temp[temp['date_dt'] == pd.Timestamp.now().normalize()]
temp.groupby(['FastTrack_DogId','date_dt']).first()
rolling_result.loc[pd.IndexSlice[:, pd.Timestamp.now().normalize()], :] = temp.groupby(['FastTrack_DogId','date_dt']).first()

```

Basically, for each greyhound we can just take the first row of data (which is correct) and set the rest of today's races to have the same value

In [19]:
# Generate rolling window features
dataset = dog_results.copy()
dataset = dataset.set_index(['FastTrack_DogId', 'date_dt']).sort_index()

# Use rolling window of 28, 91 and 365 days
rolling_windows = ['28D', '91D', '365D']
# Features to use for rolling windows calculation
features = ['RunTime_norm', 'SplitMargin_norm', 'Place_inv', 'Place_log', 'Prizemoney_norm']
# Aggregation functions to apply
aggregates = ['min', 'max', 'mean', 'median', 'std']
# Keep track of generated feature names
feature_cols = ['speed_index', 'box_win_percent']

for rolling_window in rolling_windows:
        print(f'Processing rolling window {rolling_window}')

        rolling_result = (
            dataset
            .reset_index(level=0).sort_index()
            .groupby('FastTrack_DogId')[features]
            .rolling(rolling_window)
            .agg(aggregates)
            .groupby(level=0)  # Thanks to Brett for finding this!
            .shift(1)
        )

        # My own dodgey code to work with reserve dogs
        temp = rolling_result.reset_index()
        temp = temp[temp['date_dt'] == pd.Timestamp.now().normalize()]
        temp.groupby(['FastTrack_DogId','date_dt']).first()
        rolling_result.loc[pd.IndexSlice[:, pd.Timestamp.now().normalize()], :] = temp.groupby(['FastTrack_DogId','date_dt']).first()

        # Generate list of rolling window feature names (eg: RunTime_norm_min_365D)
        agg_features_cols = [f'{f}_{a}_{rolling_window}' for f, a in itertools.product(features, aggregates)]
        # Add features to dataset
        dataset[agg_features_cols] = rolling_result
        # Keep track of generated feature names
        feature_cols.extend(agg_features_cols)

Processing rolling window 28D


c:\Users\zhoui\greyhounds_bruno\greyhound-modelling\venv_greyhounds\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Processing rolling window 91D


c:\Users\zhoui\greyhounds_bruno\greyhound-modelling\venv_greyhounds\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Processing rolling window 365D


c:\Users\zhoui\greyhounds_bruno\greyhound-modelling\venv_greyhounds\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [20]:
# Replace missing values with 0
dataset.fillna(0, inplace=True)
display(dataset.head(8))

# Only keep data after 2018-12-01
model_df = dataset.reset_index()
feature_cols = np.unique(feature_cols).tolist()
model_df = model_df[model_df['date_dt'] >= '2018-12-01']

# This line was originally part of Bruno's tutorial, but we don't run it in this script
# model_df = model_df[['date_dt', 'FastTrack_RaceId', 'DogName', 'win', 'StartPrice_probability'] + feature_cols]

# Only train model off of races where each dog has a value for each feature
races_exclude = model_df[model_df.isnull().any(axis = 1)]['FastTrack_RaceId'].drop_duplicates()
model_df = model_df[~model_df['FastTrack_RaceId'].isin(races_exclude)]

Place         DogName  Box Rug Weight  StartPrice  \
FastTrack_DogId date_dt                                                         
-2143477291     2018-09-02    7.0  YOU TELAM ANFY    1   1   31.2         5.2   
                2018-09-16    4.0  YOU TELAM ANFY    1   1   31.0        61.0   
                2018-10-07    7.0  YOU TELAM ANFY    1   1   30.5        71.0   
                2018-10-21    5.0  YOU TELAM ANFY    7   9   29.8        26.0   
                2018-11-18    5.0  YOU TELAM ANFY    1   1   30.2        41.0   
                2019-06-23    5.0  YOU TELAM ANFY    1   1   29.6        51.0   
                2019-06-30    8.0  YOU TELAM ANFY    4  10   29.5        51.0   
                2019-08-25    6.0  YOU TELAM ANFY    4   4   29.5        14.0   

                            Handicap Margin1 Margin2 PIR Checks Comments  \
FastTrack_DogId date_dt                                                    
-2143477291     2018-09-02       0.0     7.0    0.57   8      0      8.0   
                2018-09-16       0.0    10.0     5.0   8      0      8.0   
                2018-10-07       0.0    8.25     4.0   7      0      7.0   
                2018-10-21       0.0    5.25    0.29   6      0      6.0   
                2018-11-18       0.0    6.25    3.14   6      0      6.0   
                2019-06-23       0.0    5.75    0.86   8      0      8.0   
                2019-06-30       0.0   17.25     2.0   7      0      7.0   
                2019-08-25       0.0     5.0    0.57   3      0      3.0   

                            SplitMargin  RunTime  Prizemoney FastTrack_RaceId  \
FastTrack_DogId date_dt                                                         
-2143477291     2018-09-02         7.48    19.85         0.0        354469749   
                2018-09-16         7.43    19.75         0.0        360928569   
                2018-10-07         7.42    19.71         0.0        367774713   
                2018-10-21         7.46    19.85         0.0        370420123   
                2018-11-18         7.43    19.86         0.0        378695693   
                2019-06-23         7.44    19.66         0.0        445056131   
                2019-06-30        11.38    24.16         0.0        448789428   
                2019-08-25         7.33    19.72         0.0        465432748   

                           TrainerId TrainerName RaceNum  \
FastTrack_DogId date_dt                                    
-2143477291     2018-09-02      8462    A Bunney      12   
                2018-09-16      8462    A Bunney      11   
                2018-10-07      8462    A Bunney      11   
                2018-10-21      8462    A Bunney      12   
                2018-11-18      8462    A Bunney      10   
                2019-06-23      8462    A Bunney       9   
                2019-06-30      8462    A Bunney       7   
                2019-08-25      8462    A Bunney       9   

                                               RaceName RaceTime  Distance  \
FastTrack_DogId date_dt                                                      
-2143477291     2018-09-02                BRISGREYS.COM  09:01PM       331   
                2018-09-16                   SKY RACING  05:24PM       331   
                2018-10-07                   SKY RACING  08:27PM       331   
                2018-10-21              ZILLMERE SPORTS  08:55PM       331   
                2018-11-18              ZILLMERE SPORTS  08:16PM       331   
                2019-06-23   GREYHOUND ADOPTION PROGRAM  07:54PM       331   
                2019-06-30                   SKY RACING  07:45PM       395   
                2019-08-25  FABREGAS @ METICULOUS LODGE  07:40PM       331   

                                               RaceGrade        Track  \
FastTrack_DogId date_dt                                                 
-2143477291     2018-09-02  GRADE 5  PATHWAY NON-PENALTY  Albion Park   
                2018-09-16                  

---

## Generate predictions

Now this is the part that gets a bit hairy, so I am going to split it up into two parts. The good thing is that the coding will remain relatively simple.

The two things that I want to do is place live bets and save our predictions so that we can use them in a simulator we will create in the [Part V](../How_to_Automate_5).

Let's save our historical ratings for our simulator first as its quick and straight forward and then move on to placing live bets:

<!-- There are two things we are
but , the coding will all be very simple bu idea can get a bit confusing. 

This is where I am going to take a quick step back. Because not only do I want to be able to predict things live and use that to place bets I also want to generate predictions for our historic data and then put that through a simulator we will create later on. So this next part is entirely optional, if you don't want to simulate model performance you can skip to the next section.  -->



<!-- This is where I wanted to take a step back. Not only do I want to be able to 
However since historic files do not have reserve dogs/scratching, if we wanted to simulate the exchange with historic files we could just predict probabilities no

However what I actually want to do is be able to get the correct
I've spent hours working around this and the best solution I have come up with is to use the `market_catalogue` from the Betfair API to retrieve the list of dogs that will run in a race. Using the list we can select the corresponding data from our dataframe.

But this presents a second problem, because we will have multiple entries for the same dog because of reserve dogs. So then we will need select the correct one otherwise things such as box will be wrong. There are a few ways around this, you can merge on Track Name and Race Name, but the solution I have come up with is selecting the FastTrack_RaceId: -->

### Getting data ready for our simulator

Feeding our predictions through the simulator is entirely optional, but, in my opinion it is where the real sauce is made. The idea is that if we are testing our model live, we can also use the simulator to test what would happen if we tested different staking methodologies, market timings and bet placement to optimise our model. This way you can have a model but test out different strategies to optimise model performance. The thing is, I have had a play with the simulator already and we can't simulate `market_catalogue` unless you have recorded it yourself (which is what I'll be using to get `market_id` and `selection_id` to place live bets). The simulator we will use later on will only take your ratings, market_id and selection_id, so we need our data in a similar format to what we had in [How to automate III](../How_to_Automate_3). In other words, since we don't have  `market_catalogue` in the simulator, we need another way to get the market_id and selection_id.

My hacky work around is to generate the probabilities like normal (since the data is historical), we don't need to deal with reserve dogs and scratching's, then get the `market_id` and `selection_id` from the Betfair datascience greyhound model by merging on DogName and date. We can take the code we wrote in [How to automate III](../How_to_Automate_3) that downloads the greyhound ratings and convert that into a function that downloads the ratings for a date range.

In [21]:
# Generate predictions like normal
# Range of dates that we want to simulate later '2022-03-01' to '2022-04-01'
todays_data = model_df[(model_df['date_dt'] >= pd.Timestamp('2022-03-01').strftime('%Y-%m-%d')) & (model_df['date_dt'] < pd.Timestamp('2022-04-01').strftime('%Y-%m-%d'))]
dog_win_probabilities = brunos_model.predict_proba(todays_data[feature_cols])[:,1]
todays_data['prob_LogisticRegression'] = dog_win_probabilities
todays_data['renormalise_prob'] = todays_data.groupby('FastTrack_RaceId')['prob_LogisticRegression'].apply(lambda x: x / x.sum())
todays_data['rating'] = 1/todays_data['renormalise_prob']
todays_data = todays_data.sort_values(by = 'date_dt')
todays_data

C:\Users\zhoui\AppData\Local\Temp/ipykernel_25584/3121846001.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data['prob_LogisticRegression'] = dog_win_probabilities
C:\Users\zhoui\AppData\Local\Temp/ipykernel_25584/3121846001.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data['renormalise_prob'] = todays_data.groupby('FastTrack_RaceId')['prob_LogisticRegression'].apply(lambda x: x / x.sum())
C:\Users\zhoui\AppData\Local\Temp/ipykernel_25584/3121846001.py:7: SettingWithCopyWarning: 
A

,FastTrack_DogId,date_dt,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,FastTrack_RaceId,TrainerId,TrainerName,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,StartPrice_probability,win,Prizemoney_norm,Place_inv,Place_log,RunSpeed,RunTime_median,speed_index,SplitMargin_median,RunTime_norm,SplitMargin_norm,box_win_percent,RunTime_norm_min_28D,RunTime_norm_max_28D,RunTime_norm_mean_28D,RunTime_norm_median_28D,RunTime_norm_std_28D,SplitMargin_norm_min_28D,SplitMargin_norm_max_28D,SplitMargin_norm_mean_28D,SplitMargin_norm_median_28D,SplitMargin_norm_std_28D,Place_inv_min_28D,Place_inv_max_28D,Place_inv_mean_28D,Place_inv_median_28D,Place_inv_std_28D,Place_log_min_28D,Place_log_max_28D,Place_log_mean_28D,Place_log_median_28D,Place_log_std_28D,Prizemoney_norm_min_28D,Prizemoney_norm_max_28D,Prizemoney_norm_mean_28D,Prizemoney_norm_median_28D,Prizemoney_norm_std_28D,RunTime_norm_min_91D,RunTime_norm_max_91D,RunTime_norm_mean_91D,RunTime_norm_median_91D,RunTime_norm_std_91D,SplitMargin_norm_min_91D,SplitMargin_norm_max_91D,SplitMargin_norm_mean_91D,SplitMargin_norm_median_91D,SplitMargin_norm_std_91D,Place_inv_min_91D,Place_inv_max_91D,Place_inv_mean_91D,Place_inv_median_91D,Place_inv_std_91D,Place_log_min_91D,Place_log_max_91D,Place_log_mean_91D,Place_log_median_91D,Place_log_std_91D,Prizemoney_norm_min_91D,Prizemoney_norm_max_91D,Prizemoney_norm_mean_91D,Prizemoney_norm_median_91D,Prizemoney_norm_std_91D,RunTime_norm_min_365D,RunTime_norm_max_365D,RunTime_norm_mean_365D,RunTime_norm_median_365D,RunTime_norm_std_365D,SplitMargin_norm_min_365D,SplitMargin_norm_max_365D,SplitMargin_norm_mean_365D,SplitMargin_norm_median_365D,SplitMargin_norm_std_365D,Place_inv_min_365D,Place_inv_max_365D,Place_inv_mean_365D,Place_inv_median_365D,Place_inv_std_365D,Place_log_min_365D,Place_log_max_365D,Place_log_mean_365D,Place_log_median_365D,Place_log_std_365D,Prizemoney_norm_min_365D,Prizemoney_norm_max_365D,Prizemoney_norm_mean_365D,Prizemoney_norm_median_365D,Prizemoney_norm_std_365D,prob_LogisticRegression,renormalise_prob,rating
526490,523685389,2022-03-01,1.0,JOSEPH RUMBLE,8,8,30.1,2.5,0.0,8.25,0,0,0,0,0.00,21.83,1365.0,764579619,91264,B Belford,1,TAB,06:55PM,380,Novice Non Penalty,Townsville,01 Mar 22,0.318829,1,0.261288,1.000000,0.301030,0.057447,22.10,0.641825,7.56,0.561842,0.000000,0.123070,0.514985,0.514985,0.514985,0.514985,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.00,1.000000,1.000000,0.000000,0.301030,0.301030,0.301030,0.301030,0.000000,0.264578,0.264578,2.645776e-01,0.264578,0.000000,0.386805,0.514985,0.450895,0.450895,0.090637,0.482301,0.482301,0.482301,0.482301,0.000000,0.500000,1.00,0.750000,0.750000,0.353553,0.30103,0.477121,0.389076,0.389076,0.124515,0.238161,0.264578,0.251369,0.251369,0.018679,0.386805,0.514985,0.450895,0.450895,0.090637,0.482301,0.482301,0.482301,0.482301,0.000000,0.500000,1.00,0.750000,0.750000,0.353553,0.30103,0.477121,0.389076,0.389076,0.124515,0.238161,0.264578,0.251369,0.251369,0.018679,0.360558,0.321076,3.114527
494469,482776246,2022-03-01,1.0,BLAZING NENNA,4,4,25.7,2.3,0.0,3.31,0,Q/11,0,0,10.27,23.30,0.0,764625202,115912,M Delbridge,6,CHS GROUP HT3,05:37PM,410,Grade 5 Heat,Horsham,01 Mar 22,0.365634,1,0.000000,1.000000,0.301030,0.056829,23.46,0.480327,10.39,0.534335,0.558423,0.131261,0.500000,0.513187,0.504396,0.500000,0.007613,0.464830,0.633333,0.570108,0.612161,0.091786,0.250000,1.00,0.500000,0.250000,0.433013,0.301030,0.698970,0.566323,0.698970,0.229751,0.000000,0.000000,1.628327e-15,0.000000,0.000000,0.307377,0.609439,0.508348,0.524719,0.074167,0.373358,0.651515,0.543480,0.584034,0.093516,0.142857,1.00,0.535714,0.500000,0.324194,0.30103,0.903090,0.528325,0.477121,0.198314,0.000000,0.000000,0.000000,0.000000,0.000000,0.233840,0.609439,0.493044,0.512184,0.074069,0.000000,0.651515,0.515072,0.539757,0.120687,0.125000,1.00,0.443328,0.333333,0.314640,0.30103,0.954243,0.607971,0.602060,0.219526,0.000000,0.0000

In [22]:
def download_iggy_ratings(date):
    """Downloads the Betfair Iggy model ratings for a given date and formats it into a nice DataFrame.

    Args:
        date (datetime): the date we want to download the ratings for
    """
    iggy_url_1 = 'https://betfair-data-supplier-prod.herokuapp.com/api/widgets/iggy-joey/datasets?date='
    iggy_url_2 = date.strftime("%Y-%m-%d")
    iggy_url_3 = '&presenter=RatingsPresenter&csv=true'
    iggy_url = iggy_url_1 + iggy_url_2 + iggy_url_3

    # Download todays greyhounds ratings
    iggy_df = pd.read_csv(iggy_url)

    # Data clearning
    iggy_df = iggy_df.rename(
    columns={
        "meetings.races.bfExchangeMarketId":"market_id",
        "meetings.races.runners.bfExchangeSelectionId":"selection_id",
        "meetings.races.runners.ratedPrice":"rating",
        "meetings.races.number":"RaceNum",
        "meetings.name":"Track",
        "meetings.races.runners.name":"DogName"
        }
    )
    # iggy_df = iggy_df[['market_id','selection_id','rating']]
    iggy_df['market_id'] = iggy_df['market_id'].astype(str)
    iggy_df['date_dt'] = date

    # Set market_id and selection_id as index for easy referencing
    # iggy_df = iggy_df.set_index(['market_id','selection_id'])
    return(iggy_df)

# Download historical ratings over a time period and convert into a big DataFrame.
back_test_period = pd.date_range(start='2022-03-01', end='2022-04-01')
frames = [download_iggy_ratings(day) for day in back_test_period]
iggy_df = pd.concat(frames)
iggy_df

,Track,meetings.bfExchangeEventId,meetings.races.name,RaceNum,market_id,meetings.races.comment,selection_id,meetings.races.runners.number,DogName,rating,date_dt
0,Devonport,3,R1 452m Gr6,1,1.195395419,NaN,43154446,1,Magic Rogue,11.07,2022-03-01
1,Devonport,3,R1 452m Gr6,1,1.195395419,NaN,43154447,2,Youre Off,6.10,2022-03-01
2,Devonport,3,R1 452m Gr6,1,1.195395419,NaN,42352031,3,Castle Town,6.60,2022-03-01
3,Devonport,3,R1 452m Gr6,1,1.195395419,NaN,43154448,4,Buckle Up Aumond,8.02,2022-03-01
4,Devonport,3,R1 452m Gr6,1,1.195395419,NaN,42413752,5,Was That Then,17.14,2022-03-01
...,...,...,...,...,...,...,...,...,...,...,...
906,Wentworth Park,11,R9 520m Heat,9,1.196971034,NaN,27692794,3,Mercator Closer,24.49,2022-04-01
907,Wentworth Park,11,R9 520m Heat,9,1.196971034,NaN,36057267,4,Kooringa Lucy,2.61,2022-04-01
908,Wentworth Park,11,R9 520m Heat,9,1.196971034,NaN,25540155,6,Shanjo Prince,213.53,2022-04-01
909,Wentworth Park,11,R9 520m Heat,9,1.196971034,NaN,39788790,7,Lots Of Chatter,2.10,2022-04-01


In [23]:
# format DogNames to merge
todays_data['DogName'] = todays_data['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())
iggy_df['DogName'] = iggy_df['DogName'].str.upper()
# Merge
backtest = iggy_df[['market_id','selection_id','DogName','date_dt']].merge(todays_data[['rating','DogName','date_dt']], how = 'inner', on = ['DogName','date_dt'])
backtest

,market_id,selection_id,DogName,date_dt,rating
0,1.195395419,43154446,MAGIC ROGUE,2022-03-01,8.137725
1,1.195395419,43154447,YOURE OFF,2022-03-01,6.051752
2,1.195395419,42352031,CASTLE TOWN,2022-03-01,9.768546
3,1.195395419,43154448,BUCKLE UP AUMOND,2022-03-01,13.656089
4,1.195395419,42413752,WAS THAT THEN,2022-03-01,11.941057
...,...,...,...,...,...
25540,1.196921144,39309141,RANAKO BALE,2022-03-31,3.975601
25541,1.196921144,39348645,NEVAEH BALE,2022-03-31,3.917458
25542,1.196921144,26870111,INGA MIA,2022-03-31,7.459386
25543,1.196921144,42472271,WINNIE COASTER,2022-03-31,7.046953


In [24]:
# Save predictions for if we want to backtest/simulate it later
backtest.to_csv('backtest.csv', index=False) # Csv format
# backtest.to_pickle('backtest.pkl') # pickle format (faster, but can't open in excel)

Perfect, with our hacky solution we have managed to merge around a months' worth of data relatively quickly and saved it in a csv format. With all the merging it seems we have only lost around 1000 - 2000 rows of data out of 27,000 rows of data, which seems only a small price to pay.

### Getting data ready for placing live bets

Placing live bets is pretty simple but we have one issue. FastTrack Data alone is unable to tell us how many greyhounds will run in the race. For example, this race later today (2022-07-04) has 8 runners + 2 reserves:

In [34]:
todays_data[todays_data['FastTrack_RaceId'] == '798906744']

,,,FastTrack_DogId,date_dt,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,FastTrack_RaceId,TrainerId,TrainerName,RaceName,RaceTime,Distance,RaceGrade,date,StartPrice_probability,win,Prizemoney_norm,Place_inv,Place_log,RunSpeed,RunTime_median,speed_index,SplitMargin_median,RunTime_norm,SplitMargin_norm,box_win_percent,RunTime_norm_min_28D,RunTime_norm_max_28D,RunTime_norm_mean_28D,RunTime_norm_median_28D,RunTime_norm_std_28D,SplitMargin_norm_min_28D,SplitMargin_norm_max_28D,SplitMargin_norm_mean_28D,SplitMargin_norm_median_28D,SplitMargin_norm_std_28D,Place_inv_min_28D,Place_inv_max_28D,Place_inv_mean_28D,Place_inv_median_28D,Place_inv_std_28D,Place_log_min_28D,Place_log_max_28D,Place_log_mean_28D,Place_log_median_28D,Place_log_std_28D,Prizemoney_norm_min_28D,Prizemoney_norm_max_28D,Prizemoney_norm_mean_28D,Prizemoney_norm_median_28D,Prizemoney_norm_std_28D,RunTime_norm_min_91D,RunTime_norm_max_91D,RunTime_norm_mean_91D,RunTime_norm_median_91D,RunTime_norm_std_91D,SplitMargin_norm_min_91D,SplitMargin_norm_max_91D,SplitMargin_norm_mean_91D,SplitMargin_norm_median_91D,SplitMargin_norm_std_91D,Place_inv_min_91D,Place_inv_max_91D,Place_inv_mean_91D,Place_inv_median_91D,Place_inv_std_91D,Place_log_min_91D,Place_log_max_91D,Place_log_mean_91D,Place_log_median_91D,Place_log_std_91D,Prizemoney_norm_min_91D,Prizemoney_norm_max_91D,Prizemoney_norm_mean_91D,Prizemoney_norm_median_91D,Prizemoney_norm_std_91D,RunTime_norm_min_365D,RunTime_norm_max_365D,RunTime_norm_mean_365D,RunTime_norm_median_365D,RunTime_norm_std_365D,SplitMargin_norm_min_365D,SplitMargin_norm_max_365D,SplitMargin_norm_mean_365D,SplitMargin_norm_median_365D,SplitMargin_norm_std_365D,Place_inv_min_365D,Place_inv_max_365D,Place_inv_mean_365D,Place_inv_median_365D,Place_inv_std_365D,Place_log_min_365D,Place_log_max_365D,Place_log_mean_365D,Place_log_median_365D,Place_log_std_365D,Prizemoney_norm_min_365D,Prizemoney_norm_max_365D,Prizemoney_norm_mean_365D,Prizemoney_norm_median_365D,Prizemoney_norm_std_365D,prob_LogisticRegression
DogName_bf,Track,RaceNum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
YOU SEE LINA,Cannington,1,530411826,2022-07-04,0.0,YOU SEE LINA,1,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,798906744,10408,Michael McLennan,FREE ENTRY TABTOUCH PARK,01:32PM,275,Maiden,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,16.21,0.777366,5.58,0.0,0.0,0.167785,0.363135,0.419607,0.395672,0.404274,0.029202,0.395833,0.432534,0.410499,0.403130,0.019428,0.200000,0.500000,0.300000,0.200000,0.173205,0.477121,0.778151,0.677808,0.778151,0.173800,0.0,0.213623,7.120781e-02,0.000000,0.123336,0.184971,0.419607,0.324576,0.346645,0.092382,0.220812,0.432534,0.338647,0.353089,0.084555,0.166667,0.500000,0.316667,0.266667,0.153116,0.477121,0.845098,0.659617,0.690106,0.162743,0.0,0.213623,0.101436,0.092505,0.111554,0.179269,0.419607,0.303822,0.326906,0.075138,0.133803,0.432534,0.315318,0.310345,0.082022,0.125000,0.500000,0.247937,0.200000,0.121737,0.477121,0.954243,0.743299,0.778151,0.156196,0.0,0.213623,0.085118,0.000000,0.095461,0.115534
BELLA LINA,Cannington,1,547605028,2022-07-04,0.0,BELLA LINA,5,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,798906744,83951,Rodney Noden,FREE ENTRY TABTOUCH PARK,01:32PM,275,Maiden,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,16.21,0.777366,5.58,0.0,0.0,0.111111,0.228705,0.307236,0.273491,0.284534,0.040413,0.220812,0.285592,0.245073,0.228814,0.035318,0.166667,0.200000,0.188889,0.200000,0.019245,0.778151,0.845098,0.800467,0.778151,0.038652,0.0,0.000000,3.700743e-16,0.000000,0.000000,0.158046,0.307236,0.246575,0.264844,0.050600,0.029221,0.285592,0.203061,0.228814,0.091884,0.142857,0.200000,0.187075,0.200000,0.023119,0.778151,0.903090,0.805563,0.778151,0.049718,0.0,0.000000,0.000000,0.000000,0.000000,0.084276,0.307236,0.225040,0.234235,0.061784,0.029221,0.366864,0.240440,0.236842,0.088850,0.125000,0.250000,0.174702,0.166667,0.034400,0.698970,0.954243,0.835377,0.845098,0.0

If we predict probabilities and renormalise now, we will calculate incorrect probabilities.

I've spent a really long time thinking about this and testing different methods that didn't work or weren't optimal. The best solution (and least complicated) that I have come up with is to predict probabilities on the FastTrack data first. Then a few minutes before the jump when all the lineups have been confirmed we use `market_catalogue` from the Betfair API to merge our predicted probabilities, merging on `DogName`,`Track` and `RaceNum`. If we merge on these three fields, it will bypass any issues with reserve dogs and scratchings. Then we can renormalise probabilities live within Flumine.

In [26]:
# Select todays data 
todays_data = model_df[model_df['date_dt'] == pd.Timestamp.now().strftime('%Y-%m-%d')]

# Generate runner win predictions
dog_win_probabilities = brunos_model.predict_proba(todays_data[feature_cols])[:,1]
todays_data['prob_LogisticRegression'] = dog_win_probabilities

# We no longer renomralise probability in this chunk of code, do it in Flumine instead
# todays_data['renormalise_prob'] = todays_data.groupby('FastTrack_RaceId')['prob_LogisticRegression'].apply(lambda x: x / x.sum())
# todays_data['rating'] = 1/todays_data['renormalise_prob']
# todays_data = todays_data.sort_values(by = 'date_dt')

todays_data

C:\Users\zhoui\AppData\Local\Temp/ipykernel_25584/2638603781.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data['prob_LogisticRegression'] = dog_win_probabilities


,FastTrack_DogId,date_dt,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,FastTrack_RaceId,TrainerId,TrainerName,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,StartPrice_probability,win,Prizemoney_norm,Place_inv,Place_log,RunSpeed,RunTime_median,speed_index,SplitMargin_median,RunTime_norm,SplitMargin_norm,box_win_percent,RunTime_norm_min_28D,RunTime_norm_max_28D,RunTime_norm_mean_28D,RunTime_norm_median_28D,RunTime_norm_std_28D,SplitMargin_norm_min_28D,SplitMargin_norm_max_28D,SplitMargin_norm_mean_28D,SplitMargin_norm_median_28D,SplitMargin_norm_std_28D,Place_inv_min_28D,Place_inv_max_28D,Place_inv_mean_28D,Place_inv_median_28D,Place_inv_std_28D,Place_log_min_28D,Place_log_max_28D,Place_log_mean_28D,Place_log_median_28D,Place_log_std_28D,Prizemoney_norm_min_28D,Prizemoney_norm_max_28D,Prizemoney_norm_mean_28D,Prizemoney_norm_median_28D,Prizemoney_norm_std_28D,RunTime_norm_min_91D,RunTime_norm_max_91D,RunTime_norm_mean_91D,RunTime_norm_median_91D,RunTime_norm_std_91D,SplitMargin_norm_min_91D,SplitMargin_norm_max_91D,SplitMargin_norm_mean_91D,SplitMargin_norm_median_91D,SplitMargin_norm_std_91D,Place_inv_min_91D,Place_inv_max_91D,Place_inv_mean_91D,Place_inv_median_91D,Place_inv_std_91D,Place_log_min_91D,Place_log_max_91D,Place_log_mean_91D,Place_log_median_91D,Place_log_std_91D,Prizemoney_norm_min_91D,Prizemoney_norm_max_91D,Prizemoney_norm_mean_91D,Prizemoney_norm_median_91D,Prizemoney_norm_std_91D,RunTime_norm_min_365D,RunTime_norm_max_365D,RunTime_norm_mean_365D,RunTime_norm_median_365D,RunTime_norm_std_365D,SplitMargin_norm_min_365D,SplitMargin_norm_max_365D,SplitMargin_norm_mean_365D,SplitMargin_norm_median_365D,SplitMargin_norm_std_365D,Place_inv_min_365D,Place_inv_max_365D,Place_inv_mean_365D,Place_inv_median_365D,Place_inv_std_365D,Place_log_min_365D,Place_log_max_365D,Place_log_mean_365D,Place_log_median_365D,Place_log_std_365D,Prizemoney_norm_min_365D,Prizemoney_norm_max_365D,Prizemoney_norm_mean_365D,Prizemoney_norm_median_365D,Prizemoney_norm_std_365D,prob_LogisticRegression
44514,148673258,2022-07-04,0.0,SPEEDY MARINA,5,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,801455740,65928,Dawn Lee,5,LADBROKES BLENDED BETS 1-3 WIN,04:37PM,307,Grade 5,Bathurst,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,17.830,0.628105,7.880,0.0,0.0,0.136986,0.317232,0.317232,0.317232,0.317232,0.000000,0.173267,0.173267,0.173267,0.173267,0.000000,0.333333,0.333333,0.333333,0.333333,0.000000,0.602060,0.602060,6.020600e-01,0.602060,0.000000,0.212109,0.212109,2.121089e-01,0.212109,0.000000e+00,0.317232,0.332373,0.324803,0.324803,0.010706,0.173267,0.210579,0.191923,0.191923,0.026383,0.166667,0.333333,0.250000,0.250000,0.117851,0.602060,0.845098,7.235790e-01,0.723579,0.171854,0.000000,0.212109,0.106054,0.106054,0.149984,0.236982,0.371585,0.306603,0.317232,0.052095,0.173267,0.406600,0.242140,0.210579,0.096894,0.125000,0.333333,0.186905,0.166667,0.083715,0.602060,0.954243,8.299177e-01,0.845098,0.135269,0.000000,0.212109,0.042422,0.000000,0.094858,0.073607
54463,161977365,2022-07-04,0.0,FILTHY PHANTOM,7,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,801448232,110385,Tony Hinrichsen,5,GIDDY-UP (N/P) STAKE,07:27PM,342,Masters,Angle Park,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,19.675,0.533632,7.730,0.0,0.0,0.124590,0.386907,0.455919,0.427165,0.420839,0.028492,0.314004,0.500000,0.408769,0.394027,0.082256,0.166667,0.333333,0.220000,0.200000,0.064979,0.602060,0.845098,7.563224e-01,0.778151,0.090977,0.000000,0.182760,3.655208e-02,0.000000,8.173293e-02,0.386907,0.572783,0.449667,0.436026,0.053935,0.210921,0.551665,0.403088,0.394027,0.092070,0.142857,1.000000,0.291209,0.200000,0.235613,0.000000,0.903090,6.692431e-01,0.778151,0.257193,0.000000,0.249855,0.057150,0.000000,0.095131,0.277002,0.572783,0.459854,0.456897,0.059514,0.210921,0.616496,0.443040,0.443820,0.089192,0.142857,1.000000,0.464354,0.333333,0.320517,0.000000,0.903090,5.716763e-01,0.602060,0.227821,0.000000,0.249855,0.121936,0.180363,0.106520,0.128265
77950,19638404

Before we merge, let's do some minor formatting changes to the FastTrack names so we can match onto the Betfair names. Betfair excludes all apostrophes and full stops in their naming convention, so we'll create a Betfair equivalent dog name on the dataset removing these characters. We also need to do this for the tracks, sometimes FastTrack will name tracks differently to Betfair e.g., Sandown Park from Betfair is known as Sandown (SAP) in the FastTrack database.

In [27]:
# Prepare data for easy reference in flumine
todays_data['DogName_bf'] = todays_data['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())
todays_data.replace({'Sandown (SAP)': 'Sandown Park'}, regex=True, inplace=True)
todays_data = todays_data.set_index(['DogName_bf','Track','RaceNum'])
todays_data.head()

C:\Users\zhoui\AppData\Local\Temp/ipykernel_25584/90992895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todays_data['DogName_bf'] = todays_data['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())


,,,FastTrack_DogId,date_dt,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,FastTrack_RaceId,TrainerId,TrainerName,RaceName,RaceTime,Distance,RaceGrade,date,StartPrice_probability,win,Prizemoney_norm,Place_inv,Place_log,RunSpeed,RunTime_median,speed_index,SplitMargin_median,RunTime_norm,SplitMargin_norm,box_win_percent,RunTime_norm_min_28D,RunTime_norm_max_28D,RunTime_norm_mean_28D,RunTime_norm_median_28D,RunTime_norm_std_28D,SplitMargin_norm_min_28D,SplitMargin_norm_max_28D,SplitMargin_norm_mean_28D,SplitMargin_norm_median_28D,SplitMargin_norm_std_28D,Place_inv_min_28D,Place_inv_max_28D,Place_inv_mean_28D,Place_inv_median_28D,Place_inv_std_28D,Place_log_min_28D,Place_log_max_28D,Place_log_mean_28D,Place_log_median_28D,Place_log_std_28D,Prizemoney_norm_min_28D,Prizemoney_norm_max_28D,Prizemoney_norm_mean_28D,Prizemoney_norm_median_28D,Prizemoney_norm_std_28D,RunTime_norm_min_91D,RunTime_norm_max_91D,RunTime_norm_mean_91D,RunTime_norm_median_91D,RunTime_norm_std_91D,SplitMargin_norm_min_91D,SplitMargin_norm_max_91D,SplitMargin_norm_mean_91D,SplitMargin_norm_median_91D,SplitMargin_norm_std_91D,Place_inv_min_91D,Place_inv_max_91D,Place_inv_mean_91D,Place_inv_median_91D,Place_inv_std_91D,Place_log_min_91D,Place_log_max_91D,Place_log_mean_91D,Place_log_median_91D,Place_log_std_91D,Prizemoney_norm_min_91D,Prizemoney_norm_max_91D,Prizemoney_norm_mean_91D,Prizemoney_norm_median_91D,Prizemoney_norm_std_91D,RunTime_norm_min_365D,RunTime_norm_max_365D,RunTime_norm_mean_365D,RunTime_norm_median_365D,RunTime_norm_std_365D,SplitMargin_norm_min_365D,SplitMargin_norm_max_365D,SplitMargin_norm_mean_365D,SplitMargin_norm_median_365D,SplitMargin_norm_std_365D,Place_inv_min_365D,Place_inv_max_365D,Place_inv_mean_365D,Place_inv_median_365D,Place_inv_std_365D,Place_log_min_365D,Place_log_max_365D,Place_log_mean_365D,Place_log_median_365D,Place_log_std_365D,Prizemoney_norm_min_365D,Prizemoney_norm_max_365D,Prizemoney_norm_mean_365D,Prizemoney_norm_median_365D,Prizemoney_norm_std_365D,prob_LogisticRegression
DogName_bf,Track,RaceNum,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SPEEDY MARINA,Bathurst,5,148673258,2022-07-04,0.0,SPEEDY MARINA,5,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,801455740,65928,Dawn Lee,LADBROKES BLENDED BETS 1-3 WIN,04:37PM,307,Grade 5,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,17.830,0.628105,7.880,0.0,0.0,0.136986,0.317232,0.317232,0.317232,0.317232,0.000000,0.173267,0.173267,0.173267,0.173267,0.000000,0.333333,0.333333,0.333333,0.333333,0.000000,0.602060,0.602060,0.602060,0.602060,0.000000,0.212109,0.212109,2.121089e-01,0.212109,0.000000e+00,0.317232,0.332373,0.324803,0.324803,0.010706,0.173267,0.210579,0.191923,0.191923,0.026383,0.166667,0.333333,0.250000,0.250000,0.117851,0.602060,0.845098,0.723579,0.723579,0.171854,0.0,0.212109,0.106054,0.106054,0.149984,0.236982,0.371585,0.306603,0.317232,0.052095,0.173267,0.406600,0.242140,0.210579,0.096894,0.125000,0.333333,0.186905,0.166667,0.083715,0.602060,0.954243,0.829918,0.845098,0.135269,0.0,0.212109,0.042422,0.000000,0.094858,0.073607
FILTHY PHANTOM,Angle Park,5,161977365,2022-07-04,0.0,FILTHY PHANTOM,7,0,0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,801448232,110385,Tony Hinrichsen,GIDDY-UP (N/P) STAKE,07:27PM,342,Masters,04 Jul 22,0.0,0,0.0,inf,0.0,0.0,19.675,0.533632,7.730,0.0,0.0,0.124590,0.386907,0.455919,0.427165,0.420839,0.028492,0.314004,0.500000,0.408769,0.394027,0.082256,0.166667,0.333333,0.220000,0.200000,0.064979,0.602060,0.845098,0.756322,0.778151,0.090977,0.000000,0.182760,3.655208e-02,0.000000,8.173293e-02,0.386907,0.572783,0.449667,0.436026,0.053935,0.210921,0.551665,0.403088,0.394027,0.092070,0.142857,1.000000,0.291209,0.200000,0.235613,0.000000,0.903090,0.669243,0.778151,0.257193,0.0,0.249855,0.057150,0.000000,0.095131,0.277002,0.572783,0.459854,0.456897,0.059514,0.210921,0.616496,0.443040,0.443820,0.089192,0.142857,1.000000,0.464354,0.333333,0.320517,0.000000,

If you look closely at the data frame above you might notice that for reserve dogs, they will have a Box number of 9 or 10. There is only ever a max of 8 greyhounds per race therefore we will need to adjust it somehow. I didn't notice this issue for quite a while, but the good thing is the website gives us the info we need to adjust: 

![reserve_dog](hta_img/reserve_dog.png)

We can see that Rhinestone Ash is a reserve dog and has the number 9, if you click on rules, you can see what Box it is starting from: 

![reserve_dog_2](hta_img/reserve_dog_2.png)

The problem is, my webscraping is pretty poor, and it would take significant time for me to learn it. But after going through the [documentation](https://docs.developer.betfair.com/display/1smk3cen4v3lu3yomq5qye0ni/Betting+Type+Definitions#BettingTypeDefinitions-MarketDescription) again, changes to boxes are actually available through the API under the `clarifications` attribute of `marketDescription`. You will be able to access this within Flumine as `market.market_catalogue.description.clarifications`, but it's a bit weird. It returns box changes as a string that looks like this: ![box_changes](hta_img/box_changes.png)

Originally I had planned to leave this article as it is since, I've never worked with anything like this before and its already getting pretty long, however huge shoutout to Betfair Quants community and especially [Brett](https://github.com/Landrin-br) who provided his solution to working with box changes. 

In [2]:
from nltk.tokenize import regexp_tokenize
# my_string is an example string, that you will need to get live from the api via: market.market_catalogue.description.clarifications.replace("<br> Dog","<br>Dog")
my_string = "<br>Box changes:<br>Dog 9. Tralee Blaze starts from box no. 8<br><br>Dog 6. That Other One starts from box no. 2<br><br>"
print(f'HTML Comment: {my_string}')
pattern1 = r'(?<=<br>Dog ).+?(?= starts)'
pattern2 = r"(?<=\bbox no. )(\w+)"
runners_df = pd.DataFrame (regexp_tokenize(my_string, pattern1), columns = ['runner_name'])
runners_df['runner_name'] = runners_df['runner_name'].astype(str)
# Remove dog name from runner_number
runners_df['runner_number'] = runners_df['runner_name'].apply(lambda x: x[:(x.find(" ") - 1)].upper())
# Remove dog number from runner_name
runners_df['runner_name'] = runners_df['runner_name'].apply(lambda x: x[(x.find(" ") + 1):].upper())
runners_df['Box'] = regexp_tokenize(my_string, pattern2)
runners_df

HTML Comment: <br>Box changes:<br>Dog 9. Tralee Blaze starts from box no. 8<br><br>Dog 6. That Other One starts from box no. 2<br><br>


,runner_name,runner_number,Box
0,TRALEE BLAZE,9,8
1,THAT OTHER ONE,6,2


Brett's solution is amazing, there is only one problem, currently our code is structured so that we generate our predictions in the morning well before the race starts. To implement the above fix, we need to generate our predictions just before the race starts to incorporate the Box information. 

This means we need to write a little bit more code to make it happen, but we are almost there. 

So now my plan to update the old data and generate probabilities just before the race. So now just before the jump my code structure will look like this:

- pull any data on box changes from the Betfair API
- convert the box change data into a dataframe named `runners_df` using the Brett's code
- in my original dataframe named `todays_data` replace any Box data with `runners_df` data, otherwise leave it untouched
- then merge the `box_win_percent` dataframe back onto the `todays_data` dataframe 
- now we can predict probabilities again and then renormalise them

It may sound a little complicated but as we already have Brett's code there is only a few extra lines of code we need to write. This is what we will add into our Flumine strategy along with Brett's code:

```{python}
# Running Brett's code gives us a nice dataframe named runners_df that we can work with
# Replace any old Box info in our original dataframe with data available in 
runners_df = runners_df.set_index('runner_name')
todays_data.loc[(runners_df.index[runners_df.index.isin(dog_names)],track,race_number),'Box'] = runners_df.loc[runners_df.index.isin(dog_names),'Box'].to_list()
# Merge box_win_percent data onto todays_data
todays_data = todays_data.merge(box_win_percent, on=['Track', 'Distance', 'Box'], how='left')
# Merge box_win_percentage back on:
todays_data = todays_data.drop(columns = 'box_win_percentage', axis = 1)
todays_data = todays_data.merge(box_win_percent, on = ['Track', 'Distance','Box'], how = 'left')
            
# Generate probabilities using Bruno's model
todays_data.loc[(dog_names,track,race_number),'prob_LogisticRegression'] = brunos_model.predict_proba(todays_data.loc[(dog_names,track,race_number)][feature_cols])[:,1]
# renomalise probabilities
probabilities = todays_data.loc[dog_names,track,race_number]['prob_LogisticRegression']
todays_data.loc[(dog_names,track,race_number),'renormalised_prob'] = probabilities/probabilities.sum()
# convert probaiblities to ratings
todays_data.loc[(dog_names,track,race_number),'rating'] = 1/todays_data.loc[dog_names,track,race_number]['renormalised_prob']
```

Now everything is done, and we can finally move onto placing our bets

---

## Automating our predictions

Now that we have our data nicely set up. We can reference our probabilities by getting the DogName, Track and RaceNum from the Betfair polling API and then renormalised probabilities to calculate ratings with only a few lines of code. Then the rest is the same as How to [Automate III]()

In [ ]:
# Import libraries for logging in
import betfairlightweight
from flumine import Flumine, clients

# Credentials to login and logging in 
trading = betfairlightweight.APIClient('username','password',app_key='appkey')
client = clients.BetfairClient(trading, interactive_login=True)

# Login
framework = Flumine(client=client)

# Code to login when using security certificates
# trading = betfairlightweight.APIClient('username','password',app_key='appkey', certs=r'C:\Users\zhoui\openssl_certs')
# client = clients.BetfairClient(trading)

# framework = Flumine(client=client)

In [ ]:
# Import libraries and logging
from flumine import BaseStrategy 
from flumine.order.trade import Trade
from flumine.order.order import LimitOrder
from flumine.markets.market import Market
from betfairlightweight.filters import streaming_market_filter
from betfairlightweight.resources import MarketBook
import re
import pandas as pd
import numpy as np
import datetime
import logging
logging.basicConfig(filename = 'how_to_automate_4.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

Let's create a new class for our strategy called FlatBetting that finds the best available to back and lay price 60 seconds before the jump. If any of those prices have value, we will place a flat bet for $5 at those prices. This code is almost the same as [How to Automate III](../How_to_Automate_3)

Since we are now editing our `todays_data` dataframe inside our Flumine strategy we will also need to convert `todays_data` to a global variable which is a simple one liner:
```{python} 
global todays_data
```

I also wanted to call out one gotcha that, Brett found that is almost impossible to find unless you are keeping a close eye on your logs. Sometimes the polling API and streaming API doesn't match up when there are scratchings, so we need to check if it does:
```{python}
# Check the polling API and streaming API matches up (sometimes it doesn't)
if runner_cata.selection_id == runner.selection_id:
```

In [ ]:
class FlatBetting(BaseStrategy):
    def start(self) -> None:
        print("starting strategy 'FlatBetting' using the model we created the Greyhound modelling in Python Tutorial")

    def check_market_book(self, market: Market, market_book: MarketBook) -> bool:
        if market_book.status != "CLOSED":
            return True

    def process_market_book(self, market: Market, market_book: MarketBook) -> None:
        # Convert dataframe to a global variable
        global todays_data

        # At the 60 second mark:
        if market.seconds_to_start < 60 and market_book.inplay == False:
            # get the list of dog_names, name of the track/venue and race_number/RaceNum from Betfair Polling API
            dog_names = []
            track = market.market_catalogue.event.venue
            race_number = market.market_catalogue.market_name.split(' ',1)[0]  # comes out as R1/R2/R3 .. etc
            race_number = re.sub("[^0-9]", "", race_number)  # only keep the numbers 
            for runner_cata in market.market_catalogue.runners:
                dog_name = runner_cata.runner_name.split(' ',1)[1].upper()
                dog_names.append(dog_name)

            # Check if there are box changes, if there are then use Brett's code
            if market.market_catalogue.description.clarifications != None:
                # Brett's code to get Box changes:
                my_string = market.market_catalogue.description.clarifications.replace("<br> Dog","<br>Dog")
                pattern1 = r'(?<=<br>Dog ).+?(?= starts)'
                pattern2 = r"(?<=\bbox no. )(\w+)"
                runners_df = pd.DataFrame (regexp_tokenize(my_string, pattern1), columns = ['runner_name'])
                runners_df['runner_name'] = runners_df['runner_name'].astype(str)
                # Remove dog name from runner_number
                runners_df['runner_number'] = runners_df['runner_name'].apply(lambda x: x[:(x.find(" ") - 1)].upper())
                # Remove dog number from runner_name
                runners_df['runner_name'] = runners_df['runner_name'].apply(lambda x: x[(x.find(" ") + 1):].upper())
                runners_df['Box'] = regexp_tokenize(my_string, pattern2)

                # Replace any old Box info in our original dataframe with data available in runners_df
                runners_df = runners_df.set_index('runner_name')
                todays_data.loc[(runners_df.index[runners_df.index.isin(dog_names)],track,race_number),'Box'] = runners_df.loc[runners_df.index.isin(dog_names),'Box'].to_list()
                # Merge box_win_percentage back on:
                todays_data = todays_data.drop(columns = 'box_win_percentage', axis = 1)
                todays_data = todays_data.reset_index().merge(box_win_percent, on = ['Track', 'Distance','Box'], how = 'left').set_index(['DogName_bf','Track','RaceNum'])

            # Generate probabilities using Bruno's model
            todays_data.loc[(dog_names,track,race_number),'prob_LogisticRegression'] = brunos_model.predict_proba(todays_data.loc[(dog_names,track,race_number)][feature_cols])[:,1]
            # renomalise probabilities
            probabilities = todays_data.loc[dog_names,track,race_number]['prob_LogisticRegression']
            todays_data.loc[(dog_names,track,race_number),'renormalised_prob'] = probabilities/probabilities.sum()
            # convert probaiblities to ratings
            todays_data.loc[(dog_names,track,race_number),'rating'] = 1/todays_data.loc[dog_names,track,race_number]['renormalised_prob']

            # Use both the polling api (market.catalogue) and the streaming api at once:
            for runner_cata, runner in zip(market.market_catalogue.runners, market_book.runners):
                # Check the polling api and streaming api matches up (sometimes it doesn't)
                if runner_cata.selection_id == runner.selection_id:
                    # Get the dog_name from polling api then reference our data for our model rating
                    dog_name = runner_cata.runner_name.split(' ',1)[1].upper()
                    
                    # Rest is the same as How to Automate III
                    model_price = todays_data.loc[dog_name,track,race_number]['rating']
                    ### If you have an issue such as:
                        # Unknown error The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
                        # Then do model_price = todays_data.loc[dog_name,track,race_number]['rating'].item()

                    # Log info before placing bets
                    logging.info(f'dog_name: {dog_name}')
                    logging.info(f'model_price: {model_price}')
                    logging.info(f'market_id: {market_book.market_id}')
                    logging.info(f'selection_id: {runner.selection_id}')
                    
                    # If best available to back price is > rated price then flat $5 back
                    if runner.status == "ACTIVE" and runner.ex.available_to_back[0]['price'] > model_price:
                        trade = Trade(
                        market_id=market_book.market_id,
                        selection_id=runner.selection_id,
                        handicap=runner.handicap,
                        strategy=self,
                        )
                        order = trade.create_order(
                            side="BACK", order_type=LimitOrder(price=runner.ex.available_to_back[0]['price'], size=5.00)
                        )
                        market.place_order(order)
                    # If best available to lay price is < rated price then flat $5 lay
                    if runner.status == "ACTIVE" and runner.ex.available_to_lay[0]['price'] < model_price:
                        trade = Trade(
                        market_id=market_book.market_id,
                        selection_id=runner.selection_id,
                        handicap=runner.handicap,
                        strategy=self,
                        )
                        order = trade.create_order(
                            side="LAY", order_type=LimitOrder(price=runner.ex.available_to_lay[0]['price'], size=5.00)
                        )
                        market.place_order(order)

As the model we have built is a greyhound model for Australian racing let's point our strategy to Australian greyhound win markets

In [ ]:
greyhounds_strategy = FlatBetting(
    market_filter=streaming_market_filter(
        event_type_ids=["4339"], # Greyhounds markets
        country_codes=["AU"], # Australian markets
        market_types=["WIN"], # Win markets
    ),
    max_order_exposure= 50, # Max exposure per order = 50
    max_trade_count=1, # Max 1 trade per selection
    max_live_trade_count=1, # Max 1 unmatched trade per selection
)

framework.add_strategy(greyhounds_strategy)

And add our auto-terminate and bet logging from the previous tutorials:

In [ ]:
# import logging
import datetime
from flumine.worker import BackgroundWorker
from flumine.events.events import TerminationEvent

# logger = logging.getLogger(__name__)

"""
Worker can be used as followed:
    framework.add_worker(
        BackgroundWorker(
            framework,
            terminate,
            func_kwargs={"today_only": True, "seconds_closed": 1200},
            interval=60,
            start_delay=60,
        )
    )
This will run every 60s and will terminate 
the framework if all markets starting 'today' 
have been closed for at least 1200s
"""


# Function that stops automation running at the end of the day
def terminate(
    context: dict, flumine, today_only: bool = True, seconds_closed: int = 600
) -> None:
    """terminate framework if no markets
    live today.
    """
    markets = list(flumine.markets.markets.values())
    markets_today = [
        m
        for m in markets
        if m.market_start_datetime.date() == datetime.datetime.utcnow().date()
        and (
            m.elapsed_seconds_closed is None
            or (m.elapsed_seconds_closed and m.elapsed_seconds_closed < seconds_closed)
        )
    ]
    if today_only:
        market_count = len(markets_today)
    else:
        market_count = len(markets)
    if market_count == 0:
        # logger.info("No more markets available, terminating framework")
        flumine.handler_queue.put(TerminationEvent(flumine))

# Add the stopped to our framework
framework.add_worker(
    BackgroundWorker(
        framework,
        terminate,
        func_kwargs={"today_only": True, "seconds_closed": 1200},
        interval=60,
        start_delay=60,
    )
)

In [ ]:
import os
import csv
import logging
from flumine.controls.loggingcontrols import LoggingControl
from flumine.order.ordertype import OrderTypes

logger = logging.getLogger(__name__)

FIELDNAMES = [
    "bet_id",
    "strategy_name",
    "market_id",
    "selection_id",
    "trade_id",
    "date_time_placed",
    "price",
    "price_matched",
    "size",
    "size_matched",
    "profit",
    "side",
    "elapsed_seconds_executable",
    "order_status",
    "market_note",
    "trade_notes",
    "order_notes",
]


class LiveLoggingControl(LoggingControl):
    NAME = "BACKTEST_LOGGING_CONTROL"

    def __init__(self, *args, **kwargs):
        super(LiveLoggingControl, self).__init__(*args, **kwargs)
        self._setup()

    # Changed file path and checks if the file orders_hta_4.csv already exists, if it doens't then create it
    def _setup(self):
        if os.path.exists("orders_hta_4.csv"):
            logging.info("Results file exists")
        else:
            with open("orders_hta_4.csv", "w") as m:
                csv_writer = csv.DictWriter(m, delimiter=",", fieldnames=FIELDNAMES)
                csv_writer.writeheader()

    def _process_cleared_orders_meta(self, event):
        orders = event.event
        with open("orders_hta_4.csv", "a") as m:
            for order in orders:
                if order.order_type.ORDER_TYPE == OrderTypes.LIMIT:
                    size = order.order_type.size
                else:
                    size = order.order_type.liability
                if order.order_type.ORDER_TYPE == OrderTypes.MARKET_ON_CLOSE:
                    price = None
                else:
                    price = order.order_type.price
                try:
                    order_data = {
                        "bet_id": order.bet_id,
                        "strategy_name": order.trade.strategy,
                        "market_id": order.market_id,
                        "selection_id": order.selection_id,
                        "trade_id": order.trade.id,
                        "date_time_placed": order.responses.date_time_placed,
                        "price": price,
                        "price_matched": order.average_price_matched,
                        "size": size,
                        "size_matched": order.size_matched,
                        "profit": 0 if not order.cleared_order else order.cleared_order.profit,
                        "side": order.side,
                        "elapsed_seconds_executable": order.elapsed_seconds_executable,
                        "order_status": order.status.value,
                        "market_note": order.trade.market_notes,
                        "trade_notes": order.trade.notes_str,
                        "order_notes": order.notes_str,
                    }
                    csv_writer = csv.DictWriter(m, delimiter=",", fieldnames=FIELDNAMES)
                    csv_writer.writerow(order_data)
                except Exception as e:
                    logger.error(
                        "_process_cleared_orders_meta: %s" % e,
                        extra={"order": order, "error": e},
                    )

        logger.info("Orders updated", extra={"order_count": len(orders)})

    def _process_cleared_markets(self, event):
        cleared_markets = event.event
        for cleared_market in cleared_markets.orders:
            logger.info(
                "Cleared market",
                extra={
                    "market_id": cleared_market.market_id,
                    "bet_count": cleared_market.bet_count,
                    "profit": cleared_market.profit,
                    "commission": cleared_market.commission,
                },
            )

framework.add_logging_control(
    LiveLoggingControl()
)

In [ ]:
framework.run()

---

## Conclusion and next steps

Boom! We now have an automated script that will downloads all the data we need in the morning, generates a set of predictions, place flat stakes bets, logs all bets and switches itself off at the end of the day. All we need to do is hit play in the morning!

We have now written code automation code for three different strategies, however we haven't actually backtested any of our strategies or models yet. So for the final part of the How to Automate series we will be writing code to [How to simulate the Exchange to backtest and optimise our strategies](). Make sure not to miss it as this is where I believe the sauce is made (not that I have made significant sauce).

#### Complete code

Run the code from your ide by using py `<filename>`.py, making sure you amend the path to point to your input data.

[Download from Github](https://github.com/betfair-down-under/autoHubTutorials/tree/master)

In [ ]:
from joblib import load
import os
import sys

# Allow imports from src folder
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil import tz
from pandas.tseries.offsets import MonthEnd
from sklearn.preprocessing import MinMaxScaler
import itertools

import numpy as np
import pandas as pd
from nltk.tokenize import regexp_tokenize

# settings to display all columns
pd.set_option("display.max_columns", None)

import fasttrack as ft

from dotenv import load_dotenv
load_dotenv()

# Import libraries for logging in
import betfairlightweight
from flumine import Flumine, clients
# Import libraries and logging
from flumine import BaseStrategy 
from flumine.order.trade import Trade
from flumine.order.order import LimitOrder
from flumine.markets.market import Market
from betfairlightweight.filters import streaming_market_filter
from betfairlightweight.resources import MarketBook
import re
import pandas as pd
import numpy as np
import datetime
import logging
logging.basicConfig(filename = 'how_to_automate_4.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')

# import logging
from flumine.worker import BackgroundWorker
from flumine.events.events import TerminationEvent

import csv
from flumine.controls.loggingcontrols import LoggingControl
from flumine.order.ordertype import OrderTypes

logger = logging.getLogger(__name__)

brunos_model = load('logistic_regression.joblib')
brunos_model

# Validate FastTrack API connection
api_key = os.getenv('FAST_TRACK_API_KEY',)
client = ft.Fasttrack(api_key)
track_codes = client.listTracks()

# Import race data excluding NZ races
au_tracks_filter = list(track_codes[track_codes['state'] != 'NZ']['track_code'])

# Time window to import data
# First day of the month 46 months back from now
date_from = (datetime.today() - relativedelta(months=46)).replace(day=1).strftime('%Y-%m-%d')
# First day of previous month
date_to = (datetime.today() - relativedelta(months=1)).replace(day=1).strftime('%Y-%m-%d')

# Dataframes to populate data with
race_details = pd.DataFrame()
dog_results = pd.DataFrame()

# For each month, either fetch data from API or use local CSV file if we already have downloaded it
for start in pd.date_range(date_from, date_to, freq='MS'):
    start_date = start.strftime("%Y-%m-%d")
    end_date = (start + MonthEnd(1)).strftime("%Y-%m-%d")
    try:
        filename_races = f'FT_AU_RACES_{start_date}.csv'
        filename_dogs = f'FT_AU_DOGS_{start_date}.csv'

        filepath_races = f'../data/{filename_races}'
        filepath_dogs = f'../data/{filename_dogs}'

        print(f'Loading data from {start_date} to {end_date}')
        if os.path.isfile(filepath_races):
            # Load local CSV file
            month_race_details = pd.read_csv(filepath_races) 
            month_dog_results = pd.read_csv(filepath_dogs) 
        else:
            # Fetch data from API
            month_race_details, month_dog_results = client.getRaceResults(start_date, end_date, au_tracks_filter)
            month_race_details.to_csv(filepath_races, index=False)
            month_dog_results.to_csv(filepath_dogs, index=False)

        # Combine monthly data
        race_details = race_details.append(month_race_details, ignore_index=True)
        dog_results = dog_results.append(month_dog_results, ignore_index=True)
    except:
        print(f'Could not load data from {start_date} to {end_date}')

race_details.tail()

current_month_start_date = pd.Timestamp.now().replace(day=1).strftime("%Y-%m-%d")
current_month_end_date = (pd.Timestamp.now().replace(day=1)+ MonthEnd(1))
current_month_end_date = (current_month_end_date - pd.Timedelta('1 day')).strftime("%Y-%m-%d")

print(f'Start date: {current_month_start_date}')
print(f'End Date: {current_month_end_date}')

# Download data for races that have concluded this current month up untill today
# Start and end dates for current month
current_month_start_date = pd.Timestamp.now().replace(day=1).strftime("%Y-%m-%d")
current_month_end_date = (pd.Timestamp.now().replace(day=1)+ MonthEnd(1))
current_month_end_date = (current_month_end_date - pd.Timedelta('1 day')).strftime("%Y-%m-%d")

# Files names 
filename_races = f'FT_AU_RACES_{current_month_start_date}.csv'
filename_dogs = f'FT_AU_DOGS_{current_month_start_date}.csv'
# Where to store files locally
filepath_races = f'../data/{filename_races}'
filepath_dogs = f'../data/{filename_dogs}'

# Fetch data from API
month_race_details, month_dog_results = client.getRaceResults(current_month_start_date, current_month_end_date, au_tracks_filter)

# Save the files locally and replace any out of date fields
month_race_details.to_csv(filepath_races, index=False)
month_dog_results.to_csv(filepath_dogs, index=False)

dog_results

# This is super important I have spent literally hours before I found out this was causing errors
dog_results['@id'] = pd.to_numeric(dog_results['@id'])

# Append the extra data to our data frames 
race_details = race_details.append(month_race_details, ignore_index=True)
dog_results = dog_results.append(month_dog_results, ignore_index=True)

# Download the data for todays races
todays_date = pd.Timestamp.now().strftime("%Y-%m-%d")
todays_races, todays_dogs = client.getFullFormat(dt= todays_date, tracks = au_tracks_filter)

# display is for ipython notebooks only
# display(todays_races.head(1), todays_dogs.head(1))

# It seems that the todays_races dataframe doesn't have the date column, so let's add that on
todays_races['date'] = pd.Timestamp.now().strftime('%d %b %y')
todays_races.head(1)

# It also seems that in todays_dogs dataframe Box is labeled as RaceBox instead, so let's rename it
# We can also see that there are some specific dogs that have "Res." as a suffix of their name, i.e. they are reserve dogs,
# We will treat this later
todays_dogs = todays_dogs.rename(columns={"RaceBox":"Box"})
todays_dogs.tail(3)

# Appending todays data to this months data
month_dog_results = pd.concat([month_dog_results,todays_dogs],join='outer')[month_dog_results.columns]
month_race_details = pd.concat([month_race_details,todays_races],join='outer')[month_race_details.columns]

# Appending this months data to the rest of our historical data
race_details = race_details.append(month_race_details, ignore_index=True)
dog_results = dog_results.append(month_dog_results, ignore_index=True)

race_details

## Cleanse and normalise the data
# Clean up the race dataset
race_details = race_details.rename(columns = {'@id': 'FastTrack_RaceId'})
race_details['Distance'] = race_details['Distance'].apply(lambda x: int(x.replace("m", "")))
race_details['date_dt'] = pd.to_datetime(race_details['date'], format = '%d %b %y')
# Clean up the dogs results dataset
dog_results = dog_results.rename(columns = {'@id': 'FastTrack_DogId', 'RaceId': 'FastTrack_RaceId'})

# New line of code (rest of this code chunk is copied from bruno's code)
dog_results['FastTrack_DogId'] = pd.to_numeric(dog_results['FastTrack_DogId'])

# Combine dogs results with race attributes
dog_results = dog_results.merge(
    race_details, 
    how = 'left',
    on = 'FastTrack_RaceId'
)

# Convert StartPrice to probability
dog_results['StartPrice'] = dog_results['StartPrice'].apply(lambda x: None if x is None else float(x.replace('$', '').replace('F', '')) if isinstance(x, str) else x)
dog_results['StartPrice_probability'] = (1 / dog_results['StartPrice']).fillna(0)
dog_results['StartPrice_probability'] = dog_results.groupby('FastTrack_RaceId')['StartPrice_probability'].apply(lambda x: x / x.sum())

# Discard entries without results (scratched or did not finish)
dog_results = dog_results[~dog_results['Box'].isnull()]
dog_results['Box'] = dog_results['Box'].astype(int)

# Clean up other attributes
dog_results['RunTime'] = dog_results['RunTime'].astype(float)
dog_results['SplitMargin'] = dog_results['SplitMargin'].astype(float)
dog_results['Prizemoney'] = dog_results['Prizemoney'].astype(float).fillna(0)
dog_results['Place'] = pd.to_numeric(dog_results['Place'].apply(lambda x: x.replace("=", "") if isinstance(x, str) else 0), errors='coerce').fillna(0)
dog_results['win'] = dog_results['Place'].apply(lambda x: 1 if x == 1 else 0)

# Normalise some of the raw values
dog_results['Prizemoney_norm'] = np.log10(dog_results['Prizemoney'] + 1) / 12
dog_results['Place_inv'] = (1 / dog_results['Place']).fillna(0)
dog_results['Place_log'] = np.log10(dog_results['Place'] + 1).fillna(0)
dog_results['RunSpeed'] = (dog_results['RunTime'] / dog_results['Distance']).fillna(0)

## Generate features using raw data
# Calculate median winner time per track/distance
win_results = dog_results[dog_results['win'] == 1]
median_win_time = pd.DataFrame(data=win_results[win_results['RunTime'] > 0].groupby(['Track', 'Distance'])['RunTime'].median()).rename(columns={"RunTime": "RunTime_median"}).reset_index()
median_win_split_time = pd.DataFrame(data=win_results[win_results['SplitMargin'] > 0].groupby(['Track', 'Distance'])['SplitMargin'].median()).rename(columns={"SplitMargin": "SplitMargin_median"}).reset_index()
median_win_time.head()

# Calculate track speed index
median_win_time['speed_index'] = (median_win_time['RunTime_median'] / median_win_time['Distance'])
median_win_time['speed_index'] = MinMaxScaler().fit_transform(median_win_time[['speed_index']])
median_win_time.head()

# Compare dogs finish time with median winner time
dog_results = dog_results.merge(median_win_time, on=['Track', 'Distance'], how='left')
dog_results = dog_results.merge(median_win_split_time, on=['Track', 'Distance'], how='left')

# Normalise time comparison
dog_results['RunTime_norm'] = (dog_results['RunTime_median'] / dog_results['RunTime']).clip(0.9, 1.1)
dog_results['RunTime_norm'] = MinMaxScaler().fit_transform(dog_results[['RunTime_norm']])
dog_results['SplitMargin_norm'] = (dog_results['SplitMargin_median'] / dog_results['SplitMargin']).clip(0.9, 1.1)
dog_results['SplitMargin_norm'] = MinMaxScaler().fit_transform(dog_results[['SplitMargin_norm']])
dog_results.head()

# Calculate box winning percentage for each track/distance
box_win_percent = pd.DataFrame(data=dog_results.groupby(['Track', 'Distance', 'Box'])['win'].mean()).rename(columns={"win": "box_win_percent"}).reset_index()
# Add to dog results dataframe
dog_results = dog_results.merge(box_win_percent, on=['Track', 'Distance', 'Box'], how='left')
# Display example of barrier winning probabilities
print(box_win_percent.head(8))

dog_results[dog_results['FastTrack_DogId'] == 592253143].tail()[['date_dt','Place','DogName','RaceNum','Track','Distance','win','Prizemoney_norm','Place_inv','Place_log']]

# Generate rolling window features
dataset = dog_results.copy()
dataset = dataset.set_index(['FastTrack_DogId', 'date_dt']).sort_index()

# Use rolling window of 28, 91 and 365 days
rolling_windows = ['28D', '91D', '365D']
# Features to use for rolling windows calculation
features = ['RunTime_norm', 'SplitMargin_norm', 'Place_inv', 'Place_log', 'Prizemoney_norm']
# Aggregation functions to apply
aggregates = ['min', 'max', 'mean', 'median', 'std']
# Keep track of generated feature names
feature_cols = ['speed_index', 'box_win_percent']

for rolling_window in rolling_windows:
        print(f'Processing rolling window {rolling_window}')

        rolling_result = (
            dataset
            .reset_index(level=0).sort_index()
            .groupby('FastTrack_DogId')[features]
            .rolling(rolling_window)
            .agg(aggregates)
            .groupby(level=0)  # Thanks to Brett for finding this!
            .shift(1)
        )

        # My own dodgey code to work with reserve dogs
        temp = rolling_result.reset_index()
        temp = temp[temp['date_dt'] == pd.Timestamp.now().normalize()]
        temp.groupby(['FastTrack_DogId','date_dt']).first()
        rolling_result.loc[pd.IndexSlice[:, pd.Timestamp.now().normalize()], :] = temp.groupby(['FastTrack_DogId','date_dt']).first()

        # Generate list of rolling window feature names (eg: RunTime_norm_min_365D)
        agg_features_cols = [f'{f}_{a}_{rolling_window}' for f, a in itertools.product(features, aggregates)]
        # Add features to dataset
        dataset[agg_features_cols] = rolling_result
        # Keep track of generated feature names
        feature_cols.extend(agg_features_cols)

# Replace missing values with 0
dataset.fillna(0, inplace=True)
# display(dataset.head(8))  # display is only for ipython notebooks

# Only keep data after 2018-12-01
model_df = dataset.reset_index()
feature_cols = np.unique(feature_cols).tolist()
model_df = model_df[model_df['date_dt'] >= '2018-12-01']

# This line was originally part of Bruno's tutorial, but we don't run it in this script
# model_df = model_df[['date_dt', 'FastTrack_RaceId', 'DogName', 'win', 'StartPrice_probability'] + feature_cols]

# Only train model off of races where each dog has a value for each feature
races_exclude = model_df[model_df.isnull().any(axis = 1)]['FastTrack_RaceId'].drop_duplicates()
model_df = model_df[~model_df['FastTrack_RaceId'].isin(races_exclude)]

# Generate predictions like normal
# Range of dates that we want to simulate later '2022-03-01' to '2022-04-01'
todays_data = model_df[(model_df['date_dt'] >= pd.Timestamp('2022-03-01').strftime('%Y-%m-%d')) & (model_df['date_dt'] < pd.Timestamp('2022-04-01').strftime('%Y-%m-%d'))]
dog_win_probabilities = brunos_model.predict_proba(todays_data[feature_cols])[:,1]
todays_data['prob_LogisticRegression'] = dog_win_probabilities
todays_data['renormalise_prob'] = todays_data.groupby('FastTrack_RaceId')['prob_LogisticRegression'].apply(lambda x: x / x.sum())
todays_data['rating'] = 1/todays_data['renormalise_prob']
todays_data = todays_data.sort_values(by = 'date_dt')
todays_data

def download_iggy_ratings(date):
    """Downloads the Betfair Iggy model ratings for a given date and formats it into a nice DataFrame.

    Args:
        date (datetime): the date we want to download the ratings for
    """
    iggy_url_1 = 'https://betfair-data-supplier-prod.herokuapp.com/api/widgets/iggy-joey/datasets?date='
    iggy_url_2 = date.strftime("%Y-%m-%d")
    iggy_url_3 = '&presenter=RatingsPresenter&csv=true'
    iggy_url = iggy_url_1 + iggy_url_2 + iggy_url_3

    # Download todays greyhounds ratings
    iggy_df = pd.read_csv(iggy_url)

    # Data clearning
    iggy_df = iggy_df.rename(
    columns={
        "meetings.races.bfExchangeMarketId":"market_id",
        "meetings.races.runners.bfExchangeSelectionId":"selection_id",
        "meetings.races.runners.ratedPrice":"rating",
        "meetings.races.number":"RaceNum",
        "meetings.name":"Track",
        "meetings.races.runners.name":"DogName"
        }
    )
    # iggy_df = iggy_df[['market_id','selection_id','rating']]
    iggy_df['market_id'] = iggy_df['market_id'].astype(str)
    iggy_df['date_dt'] = date

    # Set market_id and selection_id as index for easy referencing
    # iggy_df = iggy_df.set_index(['market_id','selection_id'])
    return(iggy_df)

# Download historical ratings over a time period and convert into a big DataFrame.
back_test_period = pd.date_range(start='2022-03-01', end='2022-04-01')
frames = [download_iggy_ratings(day) for day in back_test_period]
iggy_df = pd.concat(frames)
iggy_df

# format DogNames to merge
todays_data['DogName'] = todays_data['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())
iggy_df['DogName'] = iggy_df['DogName'].str.upper()
# Merge
backtest = iggy_df[['market_id','selection_id','DogName','date_dt']].merge(todays_data[['rating','DogName','date_dt']], how = 'inner', on = ['DogName','date_dt'])
backtest

# Save predictions for if we want to backtest/simulate it later
backtest.to_csv('backtest.csv', index=False) # Csv format
# backtest.to_pickle('backtest.pkl') # pickle format (faster, but can't open in excel)

todays_data[todays_data['FastTrack_RaceId'] == '798906744']

# Select todays data 
todays_data = model_df[model_df['date_dt'] == pd.Timestamp.now().strftime('%Y-%m-%d')]

# Generate runner win predictions
dog_win_probabilities = brunos_model.predict_proba(todays_data[feature_cols])[:,1]
todays_data['prob_LogisticRegression'] = dog_win_probabilities

# We no longer renomralise probability in this chunk of code, do it in Flumine instead
# todays_data['renormalise_prob'] = todays_data.groupby('FastTrack_RaceId')['prob_LogisticRegression'].apply(lambda x: x / x.sum())
# todays_data['rating'] = 1/todays_data['renormalise_prob']
# todays_data = todays_data.sort_values(by = 'date_dt')

todays_data

# Prepare data for easy reference in flumine
todays_data['DogName_bf'] = todays_data['DogName'].apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())
todays_data.replace({'Sandown (SAP)': 'Sandown Park'}, regex=True, inplace=True)
todays_data = todays_data.set_index(['DogName_bf','Track','RaceNum'])
todays_data.head()

# Credentials to login and logging in 
trading = betfairlightweight.APIClient('username','password',app_key='appkey')
client = clients.BetfairClient(trading, interactive_login=True)

# Login
framework = Flumine(client=client)

# Code to login when using security certificates
# trading = betfairlightweight.APIClient('username','password',app_key='appkey', certs=r'C:\Users\zhoui\openssl_certs')
# client = clients.BetfairClient(trading)

# framework = Flumine(client=client)

class FlatBetting(BaseStrategy):
    def start(self) -> None:
        print("starting strategy 'FlatBetting' using the model we created the Greyhound modelling in Python Tutorial")

    def check_market_book(self, market: Market, market_book: MarketBook) -> bool:
        if market_book.status != "CLOSED":
            return True

    def process_market_book(self, market: Market, market_book: MarketBook) -> None:
        # Convert dataframe to a global variable
        global todays_data

        # At the 60 second mark:
        if market.seconds_to_start < 60 and market_book.inplay == False:
            # get the list of dog_names, name of the track/venue and race_number/RaceNum from Betfair Polling API
            dog_names = []
            track = market.market_catalogue.event.venue
            race_number = market.market_catalogue.market_name.split(' ',1)[0]  # comes out as R1/R2/R3 .. etc
            race_number = re.sub("[^0-9]", "", race_number)  # only keep the numbers 
            for runner_cata in market.market_catalogue.runners:
                dog_name = runner_cata.runner_name.split(' ',1)[1].upper()
                dog_names.append(dog_name)

            # Check if there are box changes, if there are then use Brett's code
            if market.market_catalogue.description.clarifications != None:
                # Brett's code to get Box changes:
                my_string = market.market_catalogue.description.clarifications.replace("<br> Dog","<br>Dog")
                pattern1 = r'(?<=<br>Dog ).+?(?= starts)'
                pattern2 = r"(?<=\bbox no. )(\w+)"
                runners_df = pd.DataFrame (regexp_tokenize(my_string, pattern1), columns = ['runner_name'])
                runners_df['runner_name'] = runners_df['runner_name'].astype(str)
                # Remove dog name from runner_number
                runners_df['runner_number'] = runners_df['runner_name'].apply(lambda x: x[:(x.find(" ") - 1)].upper())
                # Remove dog number from runner_name
                runners_df['runner_name'] = runners_df['runner_name'].apply(lambda x: x[(x.find(" ") + 1):].upper())
                runners_df['Box'] = regexp_tokenize(my_string, pattern2)

                # Replace any old Box info in our original dataframe with data available in runners_df
                runners_df = runners_df.set_index('runner_name')
                todays_data.loc[(runners_df.index[runners_df.index.isin(dog_names)],track,race_number),'Box'] = runners_df.loc[runners_df.index.isin(dog_names),'Box'].to_list()
                # Merge box_win_percentage back on:
                todays_data = todays_data.drop(columns = 'box_win_percentage', axis = 1)
                todays_data = todays_data.reset_index().merge(box_win_percent, on = ['Track', 'Distance','Box'], how = 'left').set_index(['DogName_bf','Track','RaceNum'])

            # Generate probabilities using Bruno's model
            todays_data.loc[(dog_names,track,race_number),'prob_LogisticRegression'] = brunos_model.predict_proba(todays_data.loc[(dog_names,track,race_number)][feature_cols])[:,1]
            # renomalise probabilities
            probabilities = todays_data.loc[dog_names,track,race_number]['prob_LogisticRegression']
            todays_data.loc[(dog_names,track,race_number),'renormalised_prob'] = probabilities/probabilities.sum()
            # convert probaiblities to ratings
            todays_data.loc[(dog_names,track,race_number),'rating'] = 1/todays_data.loc[dog_names,track,race_number]['renormalised_prob']

            # Use both the polling api (market.catalogue) and the streaming api at once:
            for runner_cata, runner in zip(market.market_catalogue.runners, market_book.runners):
                # Check the polling api and streaming api matches up (sometimes it doesn't)
                if runner_cata.selection_id == runner.selection_id:
                    # Get the dog_name from polling api then reference our data for our model rating
                    dog_name = runner_cata.runner_name.split(' ',1)[1].upper()
                    
                    # Rest is the same as How to Automate III
                    model_price = todays_data.loc[dog_name,track,race_number]['rating']
                    ### If you have an issue such as:
                        # Unknown error The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
                        # Then do model_price = todays_data.loc[dog_name,track,race_number]['rating'].item()

                    # Log info before placing bets
                    logging.info(f'dog_name: {dog_name}')
                    logging.info(f'model_price: {model_price}')
                    logging.info(f'market_id: {market_book.market_id}')
                    logging.info(f'selection_id: {runner.selection_id}')
                    
                    # If best available to back price is > rated price then flat $5 back
                    if runner.status == "ACTIVE" and runner.ex.available_to_back[0]['price'] > model_price:
                        trade = Trade(
                        market_id=market_book.market_id,
                        selection_id=runner.selection_id,
                        handicap=runner.handicap,
                        strategy=self,
                        )
                        order = trade.create_order(
                            side="BACK", order_type=LimitOrder(price=runner.ex.available_to_back[0]['price'], size=5.00)
                        )
                        market.place_order(order)
                    # If best available to lay price is < rated price then flat $5 lay
                    if runner.status == "ACTIVE" and runner.ex.available_to_lay[0]['price'] < model_price:
                        trade = Trade(
                        market_id=market_book.market_id,
                        selection_id=runner.selection_id,
                        handicap=runner.handicap,
                        strategy=self,
                        )
                        order = trade.create_order(
                            side="LAY", order_type=LimitOrder(price=runner.ex.available_to_lay[0]['price'], size=5.00)
                        )
                        market.place_order(order)

greyhounds_strategy = FlatBetting(
    market_filter=streaming_market_filter(
        event_type_ids=["4339"], # Greyhounds markets
        country_codes=["AU"], # Australian markets
        market_types=["WIN"], # Win markets
    ),
    max_order_exposure= 50, # Max exposure per order = 50
    max_trade_count=1, # Max 1 trade per selection
    max_live_trade_count=1, # Max 1 unmatched trade per selection
)

framework.add_strategy(greyhounds_strategy)

# logger = logging.getLogger(__name__)

"""
Worker can be used as followed:
    framework.add_worker(
        BackgroundWorker(
            framework,
            terminate,
            func_kwargs={"today_only": True, "seconds_closed": 1200},
            interval=60,
            start_delay=60,
        )
    )
This will run every 60s and will terminate 
the framework if all markets starting 'today' 
have been closed for at least 1200s
"""


# Function that stops automation running at the end of the day
def terminate(
    context: dict, flumine, today_only: bool = True, seconds_closed: int = 600
) -> None:
    """terminate framework if no markets
    live today.
    """
    markets = list(flumine.markets.markets.values())
    markets_today = [
        m
        for m in markets
        if m.market_start_datetime.date() == datetime.datetime.utcnow().date()
        and (
            m.elapsed_seconds_closed is None
            or (m.elapsed_seconds_closed and m.elapsed_seconds_closed < seconds_closed)
        )
    ]
    if today_only:
        market_count = len(markets_today)
    else:
        market_count = len(markets)
    if market_count == 0:
        # logger.info("No more markets available, terminating framework")
        flumine.handler_queue.put(TerminationEvent(flumine))

# Add the stopped to our framework
framework.add_worker(
    BackgroundWorker(
        framework,
        terminate,
        func_kwargs={"today_only": True, "seconds_closed": 1200},
        interval=60,
        start_delay=60,
    )
)

logger = logging.getLogger(__name__)

FIELDNAMES = [
    "bet_id",
    "strategy_name",
    "market_id",
    "selection_id",
    "trade_id",
    "date_time_placed",
    "price",
    "price_matched",
    "size",
    "size_matched",
    "profit",
    "side",
    "elapsed_seconds_executable",
    "order_status",
    "market_note",
    "trade_notes",
    "order_notes",
]


class LiveLoggingControl(LoggingControl):
    NAME = "BACKTEST_LOGGING_CONTROL"

    def __init__(self, *args, **kwargs):
        super(LiveLoggingControl, self).__init__(*args, **kwargs)
        self._setup()

    # Changed file path and checks if the file orders_hta_4.csv already exists, if it doens't then create it
    def _setup(self):
        if os.path.exists("orders_hta_4.csv"):
            logging.info("Results file exists")
        else:
            with open("orders_hta_4.csv", "w") as m:
                csv_writer = csv.DictWriter(m, delimiter=",", fieldnames=FIELDNAMES)
                csv_writer.writeheader()

    def _process_cleared_orders_meta(self, event):
        orders = event.event
        with open("orders_hta_4.csv", "a") as m:
            for order in orders:
                if order.order_type.ORDER_TYPE == OrderTypes.LIMIT:
                    size = order.order_type.size
                else:
                    size = order.order_type.liability
                if order.order_type.ORDER_TYPE == OrderTypes.MARKET_ON_CLOSE:
                    price = None
                else:
                    price = order.order_type.price
                try:
                    order_data = {
                        "bet_id": order.bet_id,
                        "strategy_name": order.trade.strategy,
                        "market_id": order.market_id,
                        "selection_id": order.selection_id,
                        "trade_id": order.trade.id,
                        "date_time_placed": order.responses.date_time_placed,
                        "price": price,
                        "price_matched": order.average_price_matched,
                        "size": size,
                        "size_matched": order.size_matched,
                        "profit": 0 if not order.cleared_order else order.cleared_order.profit,
                        "side": order.side,
                        "elapsed_seconds_executable": order.elapsed_seconds_executable,
                        "order_status": order.status.value,
                        "market_note": order.trade.market_notes,
                        "trade_notes": order.trade.notes_str,
                        "order_notes": order.notes_str,
                    }
                    csv_writer = csv.DictWriter(m, delimiter=",", fieldnames=FIELDNAMES)
                    csv_writer.writerow(order_data)
                except Exception as e:
                    logger.error(
                        "_process_cleared_orders_meta: %s" % e,
                        extra={"order": order, "error": e},
                    )

        logger.info("Orders updated", extra={"order_count": len(orders)})

    def _process_cleared_markets(self, event):
        cleared_markets = event.event
        for cleared_market in cleared_markets.orders:
            logger.info(
                "Cleared market",
                extra={
                    "market_id": cleared_market.market_id,
                    "bet_count": cleared_market.bet_count,
                    "profit": cleared_market.profit,
                    "commission": cleared_market.commission,
                },
            )

framework.add_logging_control(
    LiveLoggingControl()
)

framework.run()

---

## Disclaimer

Note that whilst models and automated strategies are fun and rewarding to create, we can't promise that your model or betting strategy will be profitable, and we make no representations in relation to the code shared or information on this page. If you're using this code or implementing your own strategies, you do so entirely at your own risk and you are responsible for any winnings/losses incurred. Under no circumstances will Betfair be liable for any loss or damage you suffer.